本文主要侧重embedding模型

- Reference 1: Kaggle Kernel [Code sharing, 13th place, no external data](https://www.kaggle.com/c/rossmann-store-sales/discussion/17979)
- Reference 2: Kaggle Kernel-[A Journey through Rossmann Stores](https://www.kaggle.com/omarelgabry/rossmann-store-sales/a-journey-through-rossmann-stores)
- Reference 3: Kaggle Kernel-[Predict_sales_with_pandas](https://www.kaggle.com/zygmunt/rossmann-store-sales/predict-sales-with-pandas-py)
- [XGBoost:Notes on Parameter Tuning](http://xgboost.readthedocs.io/en/latest/how_to/param_tuning.html)
- [XGBoost Parameters](http://xgboost.readthedocs.io/en/latest/parameter.html)
- [Complete Guide to Parameter Tuning in XGBoost (with codes in Python)](https://www.analyticsvidhya.com/blog/2016/03/complete-guide-parameter-tuning-xgboost-with-codes-python/)



# 训练XGBoost模型
Refers to [XGBoost Feature Importance](https://www.kaggle.com/cast42/rossmann-store-sales/xgboost-in-python-with-rmspe-v2)

Based on https://www.kaggle.com/justdoit/rossmann-store-sales/xgboost-in-python-with-rmspe/code

Public Score :  0.11389

Private Validation Score :  0.096959

#### 在运行时，每个部分可以单独运行

# 机器学习纳米学位
## 毕业项目开题报告
赵鹏举  
2017-03-16

## Rossmann药店销售额预测
### 项目背景
销售预测对于每一个企业都非常重要，机器学习的方法在其中得到了非常广泛的应用，掌握进行预测的常用方法和工作流程对于以后从事数据分析工作有着巨大的现实意义。

本项目来自Kaggle比赛[Rossman Store Sales](https://www.kaggle.com/c/rossmann-store-sales#description).  截至2015年，Rossmann在欧洲7个国家运行着超过3000家连锁药店，这些药店的营收会受到促销、竞争者、国家和学校假期、季节、地域等因素的影响。Rossmann希望参加比赛项目的选手，可以准确地地预测出位于德国1115家药店在六周内每天的销售情况；进而利用可靠的销售预测情况帮助药店经理制定更加高效的工作安排。

比赛过程中，第一名[Gert](https://kaggle2.blob.core.windows.net/forum-message-attachments/102102/3454/Rossmann_nr1_doc.pdf)在原有数据集基础上，构造出临近信息、时间信息、趋势信息等特征，并采用XGBoost方法训练模型；第三名[Cheng Guo](https://arxiv.org/pdf/1604.06737.pdf) 将原本主要用于自然语言处理的深度学习entity embedding模型应用到类别特征中，取得第三名。

### 问题描述
本问题为监督学习中的回归问题：已知1115家药店的信息以及每家药店在2年多时间内每天的销售情况，需要对接下来6周内每家药店的销售状况进行预测。回归问题的常见机器学习方法有K近邻学习、线性回归、决策树、随机森林、XGBoost、神经网络等；而实际中，为了训练出效果较好的模型，一般需要根据数据集的特点，进行特征工程，构造出有用的新特征，并对特征进行选择，同时注意防止过拟合。

在本问题提供的数据集中，销售数据作为标记值，其他属性作为特征，对选择的模型进行训练；模型的预测销售数据与标记销售数据之间的差异可以用来对模型进行评估；训练好的模型，对测试数据的预测是可以再现的。

### 输入数据
输入数据包含 train.csv和store.csv：

- train.csv是历史销售数据，每条信息包含了药店编号、日期、星期几、是否营业、是否节假日、是否促销、当日销售额以及客户数量；
- store.csv是药店补充数据，每条信息包含了点药店编号、药店类型、商品组合、最近竞争者距离及开店时间、促销有无、促销间隔和开始时间。

输入的数据对于药店销售预测是非常有用的：日期和星期几等可以提供销售额周期性的时间标定；是否节假日和促销，以及每家药店的信息和竞争者的信息，对于销售额也会有一定影响。

### 解决办法
本项目将尝试三种方法进行最终预测：

- 方法1：采用XGBoost方法；XGBoost模型是一种有监督的集成学习方法，可以直观理解为对决策树的集成，是非常有效的解决非结构化数据的方法，在Kaggle比赛中得到广泛的应用；
- 方法2：采用Entity Embedding方法；Entity Embedding属于深度学习中处理自然语言的重要方法，用来表示不同单词之间的关系，本文将用来研究不同特征之间的关系。
- 方法3：前两种方法的集成；

本项目将基于python进行实现，将会主要用到numpy、pandas、matplotlib、seaborn、sklearn、XGBoost、TensorFlow、Keras等函数库。

### 基准模型
本项目将采用两个基准模型：

- 基准一：具有相同特征参数数据子集的中位数；采用的特征包含药店编号、星期几、是否促销、药店类型、商品组合、是否节假日等；
- 基准二：Kaggle Private/Public Leader Board；Kaggle Leader Borad记录了参赛者的预测结果和最终名次，所以可用来衡量本项目能够达到怎样的最终预测结果；Public Leader Board使用了33%测试数据的预测结果，而Private Leader Board使用了67%测试数据的预测结果。

为了避免对测试数据的过拟合而导致的效果提升，本项目在完成过程中，将主要根据输入数据中划分出来的验证集对模型进行评估和优化，用Public Leader Board作为训练过程中过拟合的验证手段，仅在最终用Private Leader Board对结果进行评估。

### 评估指标
本项目采用Kaggle比赛的评估指标：RMSPE（误差百分比的均方差），可表示为
$$
RMSPE= \sqrt{\frac{1}{n}\sum_{i=1}^{n}(\frac{y_i-\hat{y_i}}{y_i})^2}
$$
其中，任何当天销售额为0的数据在评估时将被忽略； $y_i$ 表示某药店在某天的实际销售额，而$\hat{y_i}$ 表示该药店在对应这一天的预测销售额。

### 设计大纲
工作流程：

1. 数据读入：将train.csv、store.csv、test.csv读入
2. 数据可视化：针对不同的特征，对数据进行可视化；
   1. 展示不同分类特征（如星期几、节假日、促销、药店类型、商品组合等）对于销售额的影响
   2. 展示时间序列对于对于销售额的影响，比如指定店铺随着时间的销售额变化、不同因素（促销、装修、竞争等）产生的销售额变化；
   3. 观察是否有异常数据
   4. 展示不同特征之间的相关性
3. 数据整理和特征工程
   1. 对数据中缺失值、异常值进行处理
   2. 分析数据的相关性，并进行数据降维处理
   3. 进行数据聚类分析
   4. 根据需要，对数据进行清理（如归一化、采用对数处理改善分布状况）、聚合
   5. 构造更多的特征（如更详细时间信息、趋势信息等）
4. 训练基准模型
   1. 划分训练集、验证集、测试集
   2. 训练并评估基准模型
5. 训练XGBoost模型
   1. 进行特征选择，找出效果较好的特征组合；
   2. 训练不同的模型
   3. 对较好的模型进行集成
   4. 评估模型
6. 训练、评估、优化entity embedding模型
   1. 选择合适的特征，搭建entity embedding模型
   2. 搭建神经网络模型
   3. 训练、评估、优化模型及特征选择
7. 对XGBoost模型和entity embedding模型进行集成，并评估
8. 结果分析并完成报告



## 准备工作
### 0.1 调入库函数

In [1]:
#基本计算类
import pandas as pd
import numpy as np
from pandas import Series,DataFrame

#可视化
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display 

#机器学习库函数
import sklearn
from sklearn.cross_validation import train_test_split
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import make_scorer
from sklearn.ensemble import RandomForestRegressor



import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers.core import Dense,Dropout,Activation,Reshape
from keras.layers import Merge
from keras.layers.embeddings import Embedding
from keras import regularizers



import xgboost as xgb
from xgboost.sklearn import XGBRegressor




#时间类
import time
import datetime
from isoweek import Week

#文件类
import os

#其他
import itertools
import operator

#基本设定
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
%matplotlib inline
sns.set_style('whitegrid')

Using TensorFlow backend.


### 0.2 定义评估指标
本项目采用Kaggle比赛的评估指标：RMSPE（误差百分比的均方差），可表示为
$$
RMSPE= \sqrt{\frac{1}{n}\sum_{i=1}^{n}(\frac{y_i-\hat{y_i}}{y_i})^2}
$$
其中，任何当天销售额为0的数据在评估时将被忽略； $y_i$ 表示某药店在某天的实际销售额，而$\hat{y_i}$ 表示该药店在对应这一天的预测销售额。

### 0.3 Setting seed

In [2]:
seed_1=42
seed_2=43

## 3. 训练基准模型

###  3.0 将处理好的数据从本地硬盘读入

In [4]:
file_train_store_raw_df='train_store_raw_df.pickle'
# file_test_store_raw_df='test_store_raw_df.pickle'
file_feature='feature_x_list.pickle'
path='Capstone_Project_Rossman_Sales_Prediction_1'

train_store_raw_df=pd.read_pickle(os.path.join(path, file_train_store_raw_df))
# test_store_raw_df=pd.read_pickle(os.path.join(path, file_test_store_raw_df))
feature_x_list=pd.read_pickle(os.path.join(path, file_feature)).tolist()

### 3.1 从训练集中划分出验证集，基准模型将基于验证集进行度量
- Kaggle没有提供测试集数据的真实标记值, Kaggle Public/Private Leader Board适合作为衡量模型泛化能力的最终验证；当模型训练结束后，可以作为验证模型是否过拟合的工具，同时也可以参考Leader Board的结果和排名来估计模型优化的潜力；
- 本项目在完成过程中，将最后六周的数据作为验证集，用余下的训练集训练模型

In [4]:
mask_train=train_store_raw_df['Date']<'2015-06-15'
mask_valid=train_store_raw_df['Date']>='2015-06-15'
train_store_df=train_store_raw_df[mask_train]
valid_store_df=train_store_raw_df[mask_valid]
benchmark_valid_store_df=valid_store_df.copy()
# benchmark_test_store_df=test_store_raw_df.copy()

### 3.2 挑选出最优的特征组合训练基准模型

#### 3.2.1  训练基准模型
- 基准模型采用具有相同特征参数数据子集的中位数；
- 采用的特征包含药店编号、星期几，月份、是否促销、当天是否促销2等；
- 将特征组合与对应RMSPE存到dict features_RMSPE

#### 3.2.2 对验证集进行测试
- 利用选好的特征，训练出基准模型
- 将model结果以dict形式进行保存，以便于高效计算
- 使用训练好的基准模型对验证集进行测试：applymap方法可以非常高效地求出映射结果

#### 3.2.3 定义评估函数
本项目采用Kaggle比赛的评估指标：RMSPE（误差百分比的均方差），可表示为
$$
RMSPE= \sqrt{\frac{1}{n}\sum_{i=1}^{n}(\frac{y_i-\hat{y_i}}{y_i})^2}
$$
其中，任何当天销售额为0的数据在评估时将被忽略； $y_i$ 表示某药店在某天的实际销售额，而$\hat{y_i}$ 表示该药店在对应这一天的预测销售额。

#### 3.2.4 求出基准模型在验证集上的RMSPE

In [5]:
#使用训练好的基准模型对验证集进行测试：applymap方法可以非常高效地求出映射结果
def predict_benchmark_model(df,features,map_dict):
    mask_0=df['Open']==0
    mask_1=df['Open']!=0
    df.loc[mask_0,'Sales_Predicted']=0
    df['tuple_map']=df[features].apply(tuple,axis=1)
    df.loc[mask_1,'Sales_Predicted']=df.loc[mask_1,'tuple_map'].map(map_dict)
    return df

#定义评估函数
def calc_RMSPE(input_df,true_label='Sales',predicted_label='Sales_Predicted'):
    mask=input_df[true_label]>0
    sale=input_df.loc[mask,[true_label,predicted_label]]
    sale['errror']=np.power((sale[true_label]-sale[predicted_label])/sale[true_label],2.0)
    return np.sqrt(np.sum(sale['errror'])/(len(sale)))

#采用的特征包含药店编号、星期几，月份、是否促销、当天是否促销2等；
features_benchmark=(['Store','DayOfWeek','Promo','Month','InPromo2Today'])
#将特征组合与对应RMSPE存到dict features_RMSPE
features_RMSPE={}

#提取出所有可能的特征组合放入subset
for L in range(2, len(features_benchmark)+1):
    for subset in itertools.combinations(features_benchmark, L):
        valid_df=benchmark_valid_store_df.copy()
        
        features=list(subset)
        features_sales=features+['Sales']
        
        #利用选好的特征，训练出基准模型
        benchmark_model=train_store_df[train_store_df['Open']==1][features_sales].groupby(features).median()
        #将model结果以dict形式进行保存，以便于高效计算
        benchmark_model_dict=benchmark_model['Sales'].to_dict()
        
        valid_df= predict_benchmark_model(valid_df,features,benchmark_model_dict)   
        RMSPE_valid_benchmark=calc_RMSPE(valid_df)
        features_RMSPE[subset]=RMSPE_valid_benchmark

#### 3.2.5 打印features_RMSPE中数值较小的三个

In [6]:
#打印features_RMSPE中数值较小的三个
sorted_RMSPE = sorted(features_RMSPE.items(), key=operator.itemgetter(1))
print("-----------RMSPE较小的三个特征组合与在对应验证集的RMSPE-------------")
for ii in range(3):
    print sorted_RMSPE[ii][0],':',sorted_RMSPE[ii][1]

-----------RMSPE较小的三个特征组合与在对应验证集的RMSPE-------------
('Store', 'DayOfWeek', 'Promo', 'InPromo2Today') : 0.140907493872
('Store', 'DayOfWeek', 'Promo') : 0.141226490165
('Store', 'DayOfWeek', 'Promo', 'Month', 'InPromo2Today') : 0.14656682642


In [7]:
####3.3 训练基准模型，并在测试集上进行预测，并提交到Kaggle查看结果
#- 把RMSPE最小的特征赋给features_benchmark，并使用所有的train_store_raw_df数据训练模型

### 3.3 利用挑选的特征训练基准模型，并在验证集上预测

In [8]:
features_benchmark=list(min(features_RMSPE, key=features_RMSPE.get))
features_benchmark_sales=features_benchmark+['Sales']

#利用选好的特征，训练出基准模型
benchmark_model=train_store_df[train_store_df['Open']==1][features_benchmark_sales].groupby(features_benchmark).median()
#将model结果以dict形式进行保存，以便于高效计算
benchmark_model_dict=benchmark_model['Sales'].to_dict()
        
valid_df= predict_benchmark_model(valid_df,features_benchmark,benchmark_model_dict)   
RMSPE_valid_benchmark=calc_RMSPE(valid_df)
print('基准模型在验证集上的RMSPE:{:.6f}'.format(RMSPE_valid_benchmark))

# benchmark_model=train_store_raw_df[train_store_raw_df['Open']==1][features_benchmark_sales].groupby(features_benchmark).median()
# benchmark_model_dict=benchmark_model['Sales'].to_dict()

# benchmark_test_store_df= predict_benchmark_model(benchmark_test_store_df,features_benchmark,benchmark_model_dict)   

# benchmark_test_output=benchmark_test_store_df.ix[:,['Id','Sales_Predicted']].rename(columns={'Sales_Predicted': 'Sales'})
# benchmark_test_output.sort_values(by='Id',inplace=True)
# path='Capstone_Project_Rossman_Sales_Prediction_1'
# filename='benchmark_model_test_'+'features_'+'_'.join(features_benchmark)+'.csv'
# benchmark_test_output.to_csv(os.path.join(path, filename),index=False)

基准模型在验证集上的RMSPE:0.140907


In [9]:
# #### [Kaggle Leader Board Result](https://www.kaggle.com/c/rossmann-store-sales/submissions):

# |    Kaggle Leader Board   | Result |
# | ---------- | --- |
# | Private |  0.14807 |
# | Public       |  0.14044 |



## 5. 搭建Embedding模型，并预测结果


###  5.0数据读入及预处理
#### 5.0.1 将处理好的数据从本地硬盘读入

In [3]:
file_train_store_raw_df='train_store_raw_df.pickle'
# file_test_store_raw_df='test_store_raw_df.pickle'
file_feature='feature_x_list.pickle'
path='Capstone_Project_Rossman_Sales_Prediction_1'

train_store_raw_df=pd.read_pickle(os.path.join(path, file_train_store_raw_df))
# test_store_raw_df=pd.read_pickle(os.path.join(path, file_test_store_raw_df))
feature_x_list=pd.read_pickle(os.path.join(path, file_feature)).tolist()

#### 5.0.2 每个特征所对应的unique数值
- dict_feature_range：每个feature所对应的范围
- dict_feature_offset：每个feature最小值移到0所对应的offset

In [4]:
dict_feature_range={}
dict_feature_offset={}

for ii in feature_x_list:
    unique_list=train_store_raw_df[ii].unique()
    max_v=max(unique_list)
    min_v=min(unique_list)
    dict_feature_range[ii]=(int)(max_v-min_v+1)
    dict_feature_offset[ii]=(int)(min_v)
    print '{0: <30}'.format(ii),':','unique=','{0: <5}'.format(len(unique_list)),\
    ',max=','{0: <5}'.format((max_v)),',min=','{0: <5}'.format((min_v)), \
    ',range=','{0: <5}'.format((dict_feature_range[ii])),',offset=','{0: <5}'.format((dict_feature_offset[ii]))
    


#     print ii,':','unique=',len(unique_list),',max=',max_v,',min=',min_v, \
#     ',range=',dict_feature_range[ii],',offset=',dict_feature_offset[ii]

Store                          : unique= 1115  ,max= 1115  ,min= 1     ,range= 1115  ,offset= 1    
DayOfWeek                      : unique= 7     ,max= 7     ,min= 1     ,range= 7     ,offset= 1    
Year                           : unique= 3     ,max= 2015  ,min= 2013  ,range= 3     ,offset= 2013 
Month                          : unique= 12    ,max= 12    ,min= 1     ,range= 12    ,offset= 1    
Day                            : unique= 31    ,max= 31    ,min= 1     ,range= 31    ,offset= 1    
DayOfYear                      : unique= 365   ,max= 365   ,min= 1     ,range= 365   ,offset= 1    
StoreType_cat                  : unique= 4     ,max= 3     ,min= 0     ,range= 4     ,offset= 0    
Assortment_cat                 : unique= 3     ,max= 2     ,min= 0     ,range= 3     ,offset= 0    
StateHoliday_cat               : unique= 4     ,max= 3     ,min= 0     ,range= 4     ,offset= 0    
SchoolHoliday                  : unique= 2     ,max= 1     ,min= 0     ,range= 2     ,offset= 0    


#### 5.0.3 将数据对应feature_x_list每列添加offset，将数据的最小值设为0

In [5]:
modified_train_store_raw_df=train_store_raw_df.copy()
# modified_test_store_raw_df=test_store_raw_df.copy()
for col in feature_x_list:
    modified_train_store_raw_df[col]=modified_train_store_raw_df[col]-dict_feature_offset[col]
#     modified_test_store_raw_df[col]=modified_test_store_raw_df[col]-dict_feature_offset[col]
    
for ii in feature_x_list:
    unique_list=modified_train_store_raw_df[ii].unique()
    max_v=max(unique_list)
    min_v=min(unique_list)
    print '{0: <30}'.format(ii),':','unique=','{0: <5}'.format(len(unique_list)),\
    ',max=','{0: <5}'.format((max_v)),',min=','{0: <5}'.format((min_v)), \
    ',range=','{0: <5}'.format((int)(max_v-min_v+1)),',offset=','{0: <5}'.format((int)(min_v))    

Store                          : unique= 1115  ,max= 1114  ,min= 0     ,range= 1115  ,offset= 0    
DayOfWeek                      : unique= 7     ,max= 6     ,min= 0     ,range= 7     ,offset= 0    
Year                           : unique= 3     ,max= 2     ,min= 0     ,range= 3     ,offset= 0    
Month                          : unique= 12    ,max= 11    ,min= 0     ,range= 12    ,offset= 0    
Day                            : unique= 31    ,max= 30    ,min= 0     ,range= 31    ,offset= 0    
DayOfYear                      : unique= 365   ,max= 364   ,min= 0     ,range= 365   ,offset= 0    
StoreType_cat                  : unique= 4     ,max= 3     ,min= 0     ,range= 4     ,offset= 0    
Assortment_cat                 : unique= 3     ,max= 2     ,min= 0     ,range= 3     ,offset= 0    
StateHoliday_cat               : unique= 4     ,max= 3     ,min= 0     ,range= 4     ,offset= 0    
SchoolHoliday                  : unique= 2     ,max= 1     ,min= 0     ,range= 2     ,offset= 0    


#### 5.0.4 构造train_df数据，valid_df数据

In [6]:
mask_train=(modified_train_store_raw_df['Date']<'2015-06-15') &(modified_train_store_raw_df['Outlier_3']==False) \
            & (modified_train_store_raw_df['Open']==1) & (modified_train_store_raw_df['Sales']>0)
mask_valid=(modified_train_store_raw_df['Date']>='2015-06-15') & (modified_train_store_raw_df['Open']==1)\
            & (modified_train_store_raw_df['Sales']>0)#&(modified_train_store_raw_df['Outlier_3']==False)

df_train=modified_train_store_raw_df.loc[mask_train,feature_x_list]
df_valid=modified_train_store_raw_df.loc[mask_valid,feature_x_list]
y_train_data=np.array(modified_train_store_raw_df.loc[mask_train,'Sales'])
y_valid_data=np.array(modified_train_store_raw_df.loc[mask_valid,'Sales'])

# df_test=modified_test_store_raw_df.loc[modified_test_store_raw_df['Open']==1]

#### 5.0.5 构造embedding模型所需要的train, valid数据

In [7]:
x_train_data=[]
x_valid_data=[]
for ii in feature_x_list:
    x_train_data.append(np.array(df_train[ii]))
    x_valid_data.append(np.array(df_valid[ii]))  

### 5.1 搭建embedding模型
- 构造list models，将每个feature对应的embedding装进去
    - 输入range: dict_feature_range
    - 对应的embedding数目:calc_embedding_space

In [71]:
def calc_embedding_space(x):
    dict_map={
        1115:50,
        365:30,
        31:10,
        16:10,
        15:10,
        12:8,
        7:6,
        4:3,        
        3:2,
        2:1,
        1:1
    }
    return dict_map[x]


models=[]
for ii in feature_x_list:
    input_range=dict_feature_range[ii]
    
#     if input_range ==2 :
#         temp_name='Dense_'+ii
#         model= Sequential(name=temp_name)
#         model.add(Dense(1, input_dim=1))
#         models.append(model)
#     else:    
    embedding_space=calc_embedding_space(input_range)        
    temp_name='Embedding_'+ii
    model=Sequential(name=temp_name)
    model.add(Embedding(input_range,embedding_space,input_length=1))
    model.add(Reshape((embedding_space, ), input_shape=(1,embedding_space)))
    models.append(model)


    
def create_embedding_model():
    dropout_rate=0.1

    embedding_model=Sequential()
    embedding_model.add(Merge(models,mode='concat',concat_axis=-1))

    embedding_model.add(Dense(512,kernel_initializer='uniform'))
    embedding_model.add(Activation('relu'))
    embedding_model.add(Dense(128,kernel_initializer='uniform'))
    embedding_model.add(Activation('relu'))
    embedding_model.add(Dense(32,kernel_initializer='uniform'))
    embedding_model.add(Activation('relu'))
    embedding_model.add(Dropout(dropout_rate))
    embedding_model.add(Dense(1))
    embedding_model.add(Activation('sigmoid'))

    embedding_model.compile(loss='mean_absolute_error',optimizer='adam')
    return embedding_model

###  5.2 训练模型

In [72]:
max_log_y=np.max(np.log(y_train_data))

def _val_for_fit(val):
    val=np.log(val)
    return val/max_log_y
def _val_for_pred(val):
    return np.exp(val*max_log_y)

def calc_predict_rmspe(trained_model,x_data,y_data):
    flag=y_data>0
    predicted_value=trained_model.predict( x_data).flatten()
    return np.sqrt(np.mean(np.power((_val_for_pred(predicted_value[flag])-y_data[flag])/y_data[flag],2.0)))
    
    

nb_epoch=10

embedding_model=create_embedding_model()
embedding_model.fit(x_train_data, _val_for_fit(y_train_data), 
                    validation_data=(x_valid_data,_val_for_fit(y_valid_data)),
                    epochs=nb_epoch, 
                    batch_size=256
                           # callbacks=[self.checkpointer],
                )
valid_rmspe=calc_predict_rmspe(embedding_model,x_valid_data,y_valid_data)
train_rmspe=calc_predict_rmspe(embedding_model,x_train_data,y_train_data)
print "train_rmspe :{:f},\tvalid_rmspe:{:f}".format(train_rmspe,valid_rmspe)



Train on 786180 samples, validate on 45852 samples
Epoch 1/10
786180/786180 [==============================] - 36s - loss: 0.0173 - val_loss: 0.0114
Epoch 2/10
786180/786180 [==============================] - 36s - loss: 0.0092 - val_loss: 0.0100
Epoch 3/10
786180/786180 [==============================] - 37s - loss: 0.0076 - val_loss: 0.0097
Epoch 4/10
786180/786180 [==============================] - 37s - loss: 0.0072 - val_loss: 0.0097
Epoch 5/10
786180/786180 [==============================] - 36s - loss: 0.0070 - val_loss: 0.0093
Epoch 6/10
786180/786180 [==============================] - 36s - loss: 0.0068 - val_loss: 0.0091
Epoch 7/10
786180/786180 [==============================] - 36s - loss: 0.0066 - val_loss: 0.0091
Epoch 8/10
786180/786180 [==============================] - 37s - loss: 0.0065 - val_loss: 0.0092
Epoch 9/10
786180/786180 [==============================] - 36s - loss: 0.0064 - val_loss: 0.0092
Epoch 10/10
786180/786180 [==============================] - 37s - 

In [77]:
max_log_y=np.max(np.log(y_train_data))

def _val_for_fit(val):
    val=np.log(val)
    return val/max_log_y
def _val_for_pred(val):
    return np.exp(val*max_log_y)

def calc_predict_rmspe(trained_model,x_data,y_data):
    flag=y_data>0
    predicted_value=trained_model.predict( x_data).flatten()
    return np.sqrt(np.mean(np.power((_val_for_pred(predicted_value[flag])-(y_data[flag]))/y_data[flag],2.0)))
    
    

nb_epoch=15

embedding_model=create_embedding_model()
embedding_model.fit(x_train_data, _val_for_fit(y_train_data), 
                    validation_data=(x_valid_data,_val_for_fit(y_valid_data)),
                    epochs=nb_epoch, 
                    batch_size=256
                           # callbacks=[self.checkpointer],
                )
valid_rmspe=calc_predict_rmspe(embedding_model,x_valid_data,y_valid_data)
train_rmspe=calc_predict_rmspe(embedding_model,x_train_data,y_train_data)
print "train_rmspe :{:f},\tvalid_rmspe:{:f}".format(train_rmspe,valid_rmspe)



Train on 786180 samples, validate on 45852 samples
Epoch 1/15
786180/786180 [==============================] - 31s - loss: 0.0147 - val_loss: 0.0095
Epoch 2/15
786180/786180 [==============================] - 29s - loss: 0.0077 - val_loss: 0.0102
Epoch 3/15
786180/786180 [==============================] - 31s - loss: 0.0066 - val_loss: 0.0097
Epoch 4/15
786180/786180 [==============================] - 35s - loss: 0.0064 - val_loss: 0.0100
Epoch 5/15
786180/786180 [==============================] - 33s - loss: 0.0063 - val_loss: 0.0106
Epoch 6/15
786180/786180 [==============================] - 33s - loss: 0.0062 - val_loss: 0.0101
Epoch 7/15
786180/786180 [==============================] - 32s - loss: 0.0061 - val_loss: 0.0102
Epoch 8/15
786180/786180 [==============================] - 30s - loss: 0.0060 - val_loss: 0.0103
Epoch 9/15
786180/786180 [==============================] - 29s - loss: 0.0060 - val_loss: 0.0098
Epoch 10/15
786180/786180 [==============================] - 30s - 

### 思路：重写loss函数
- 使用其他函数作为loss函数，

In [33]:
def calc_embedding_space(x):
    dict_map={
        1115:50,
        365:30,
        31:10,
        16:10,
        15:10,
        12:8,
        7:6,
        4:3,        
        3:2,
        2:1,
        1:1
    }
    return dict_map[x]


models=[]
for ii in feature_x_list:
    input_range=dict_feature_range[ii]
    
#     if input_range ==2 :
#         temp_name='Dense_'+ii
#         model= Sequential(name=temp_name)
#         model.add(Dense(1, input_dim=1))
#         models.append(model)
#     else:    
    embedding_space=calc_embedding_space(input_range)        
    temp_name='Embedding_'+ii
    model=Sequential(name=temp_name)
    model.add(Embedding(input_range,embedding_space,input_length=1))
    model.add(Reshape((embedding_space, ), input_shape=(1,embedding_space)))
    models.append(model)

    
max_log_y=np.max(np.log(y_train_data))

def _val_for_fit(val):
    val=np.log(val)
    return val/max_log_y

def _val_for_pred(val):
    return tf.exp(val*max_log_y)        

def loss_func_rmspe(y_true,y_pred):
    return tf.sqrt(tf.reduce_mean(tf.pow((_val_for_pred(y_pred)-_val_for_pred(y_true))/_val_for_pred(y_true),2.0)))

def calc_predict_rmspe(trained_model,x_data,y_data):
    predicted_value=trained_model.predict( x_data).flatten()
    return np.sqrt(np.mean(np.power((np.exp(predicted_value*max_log_y)-y_data.astype(np.float32))/y_data.astype(np.float32),2.0)))    


        
def create_embedding_model():
    dropout_rate=0.2

    embedding_model=Sequential()
    embedding_model.add(Merge(models,mode='concat',concat_axis=-1))

    embedding_model.add(Dense(512,kernel_initializer='uniform'))
    embedding_model.add(Activation('relu'))
    embedding_model.add(Dense(128,kernel_initializer='uniform'))
    embedding_model.add(Activation('relu'))
    embedding_model.add(Dense(32,kernel_initializer='uniform'))
    embedding_model.add(Activation('relu'))
    embedding_model.add(Dropout(dropout_rate))
    embedding_model.add(Dense(1))
    embedding_model.add(Activation('sigmoid'))

    embedding_model.compile(loss=loss_func_rmspe,optimizer='adam')
    return embedding_model


In [34]:
nb_epoch=1

embedding_model=create_embedding_model()
embedding_model.fit(x_train_data, _val_for_fit(y_train_data), 
                    validation_data=(x_valid_data,_val_for_fit(y_valid_data)),
                    epochs=nb_epoch, 
                    batch_size=256
                           # callbacks=[self.checkpointer],
                )

valid_rmspe=calc_predict_rmspe(embedding_model,x_valid_data,y_valid_data)
train_rmspe=calc_predict_rmspe(embedding_model,x_train_data,y_train_data)
print "train_rmspe :{:f},\tvalid_rmspe:{:f}".format(train_rmspe,valid_rmspe)

Train on 786180 samples, validate on 45852 samples
Epoch 1/1
786180/786180 [==============================] - 31s - loss: 0.2328 - val_loss: 0.1566
train_rmspe :0.158865,	valid_rmspe:0.158396


In [35]:
nb_epoch=2

embedding_model=create_embedding_model()
embedding_model.fit(x_train_data, _val_for_fit(y_train_data), 
                    validation_data=(x_valid_data,_val_for_fit(y_valid_data)),
                    epochs=nb_epoch, 
                    batch_size=256
                           # callbacks=[self.checkpointer],
                )

valid_rmspe=calc_predict_rmspe(embedding_model,x_valid_data,y_valid_data)
train_rmspe=calc_predict_rmspe(embedding_model,x_train_data,y_train_data)
print "train_rmspe :{:f},\tvalid_rmspe:{:f}".format(train_rmspe,valid_rmspe)

Train on 786180 samples, validate on 45852 samples
Epoch 1/2
786180/786180 [==============================] - 31s - loss: 0.2111 - val_loss: 0.1504
Epoch 2/2
786180/786180 [==============================] - 31s - loss: 0.1295 - val_loss: 0.1526
train_rmspe :0.131919,	valid_rmspe:0.154768


In [36]:
nb_epoch=3

embedding_model=create_embedding_model()
embedding_model.fit(x_train_data, _val_for_fit(y_train_data), 
                    validation_data=(x_valid_data,_val_for_fit(y_valid_data)),
                    epochs=nb_epoch, 
                    batch_size=256
                           # callbacks=[self.checkpointer],
                )

valid_rmspe=calc_predict_rmspe(embedding_model,x_valid_data,y_valid_data)
train_rmspe=calc_predict_rmspe(embedding_model,x_train_data,y_train_data)
print "train_rmspe :{:f},\tvalid_rmspe:{:f}".format(train_rmspe,valid_rmspe)

Train on 786180 samples, validate on 45852 samples
Epoch 1/3
786180/786180 [==============================] - 30s - loss: 0.2110 - val_loss: 0.1518
Epoch 2/3
786180/786180 [==============================] - 30s - loss: 0.1201 - val_loss: 0.1542
Epoch 3/3
786180/786180 [==============================] - 31s - loss: 0.1155 - val_loss: 0.1466
train_rmspe :0.117898,	valid_rmspe:0.148456


In [37]:
nb_epoch=4

embedding_model=create_embedding_model()
embedding_model.fit(x_train_data, _val_for_fit(y_train_data), 
                    validation_data=(x_valid_data,_val_for_fit(y_valid_data)),
                    epochs=nb_epoch, 
                    batch_size=256
                           # callbacks=[self.checkpointer],
                )

valid_rmspe=calc_predict_rmspe(embedding_model,x_valid_data,y_valid_data)
train_rmspe=calc_predict_rmspe(embedding_model,x_train_data,y_train_data)
print "train_rmspe :{:f},\tvalid_rmspe:{:f}".format(train_rmspe,valid_rmspe)

Train on 786180 samples, validate on 45852 samples
Epoch 1/4
786180/786180 [==============================] - 30s - loss: 0.2089 - val_loss: 0.1489
Epoch 2/4
786180/786180 [==============================] - 30s - loss: 0.1268 - val_loss: 0.1477
Epoch 3/4
786180/786180 [==============================] - 31s - loss: 0.1144 - val_loss: 0.1390
Epoch 4/4
786180/786180 [==============================] - 31s - loss: 0.1116 - val_loss: 0.1424
train_rmspe :0.106875,	valid_rmspe:0.144426


In [38]:
nb_epoch=5

embedding_model=create_embedding_model()
embedding_model.fit(x_train_data, _val_for_fit(y_train_data), 
                    validation_data=(x_valid_data,_val_for_fit(y_valid_data)),
                    epochs=nb_epoch, 
                    batch_size=256
                           # callbacks=[self.checkpointer],
                )

valid_rmspe=calc_predict_rmspe(embedding_model,x_valid_data,y_valid_data)
train_rmspe=calc_predict_rmspe(embedding_model,x_train_data,y_train_data)
print "train_rmspe :{:f},\tvalid_rmspe:{:f}".format(train_rmspe,valid_rmspe)

Train on 786180 samples, validate on 45852 samples
Epoch 1/5
786180/786180 [==============================] - 30s - loss: 0.2091 - val_loss: 0.1385
Epoch 2/5
786180/786180 [==============================] - 30s - loss: 0.1172 - val_loss: 0.1415
Epoch 3/5
786180/786180 [==============================] - 31s - loss: 0.1136 - val_loss: 0.1539
Epoch 4/5
786180/786180 [==============================] - 31s - loss: 0.1099 - val_loss: 0.1413
Epoch 5/5
786180/786180 [==============================] - 31s - loss: 0.1064 - val_loss: 0.1356
train_rmspe :0.102818,	valid_rmspe:0.137644


### 思路：重写loss函数
- 使用其他函数作为loss函数，

In [42]:
def calc_embedding_space(x):
    dict_map={
        1115:50,
        365:30,
        31:10,
        16:10,
        15:10,
        12:8,
        7:6,
        4:3,        
        3:2,
        2:1,
        1:1
    }
    return dict_map[x]


models=[]
for ii in feature_x_list:
    input_range=dict_feature_range[ii]
    
#     if input_range ==2 :
#         temp_name='Dense_'+ii
#         model= Sequential(name=temp_name)
#         model.add(Dense(1, input_dim=1))
#         models.append(model)
#     else:    
    embedding_space=calc_embedding_space(input_range)        
    temp_name='Embedding_'+ii
    model=Sequential(name=temp_name)
    model.add(Embedding(input_range,embedding_space,input_length=1))
    model.add(Reshape((embedding_space, ), input_shape=(1,embedding_space)))
    models.append(model)

    
max_log_y=np.max(np.log(y_train_data))

def _val_for_fit(val):
    val=np.log(val)
    return val/max_log_y

def _val_for_pred(val):
    return tf.exp(val*max_log_y)        

def loss_func_rmspe(y_true,y_pred):
    return tf.sqrt(tf.reduce_mean(tf.pow((_val_for_pred(y_pred)-_val_for_pred(y_true))/_val_for_pred(y_true),2.0)))

def calc_predict_rmspe(trained_model,x_data,y_data):
    predicted_value=trained_model.predict( x_data).flatten()
    return np.sqrt(np.mean(np.power((np.exp(predicted_value*max_log_y)-y_data.astype(np.float32))/y_data.astype(np.float32),2.0)))    
        
def create_embedding_model():
    dropout_rate=0.2

    embedding_model=Sequential()
    embedding_model.add(Merge(models,mode='concat',concat_axis=-1))

    embedding_model.add(Dense(1024,kernel_initializer='uniform'))
    embedding_model.add(Activation('relu'))
    embedding_model.add(Dense(256,kernel_initializer='uniform'))
    embedding_model.add(Activation('relu'))
    embedding_model.add(Dense(64,kernel_initializer='uniform'))
    embedding_model.add(Activation('relu'))
    embedding_model.add(Dropout(dropout_rate))
    embedding_model.add(Dense(1))
    embedding_model.add(Activation('sigmoid'))

    embedding_model.compile(loss=loss_func_rmspe,optimizer='adam')
    return embedding_model


In [40]:
nb_epoch=1

embedding_model=create_embedding_model()
embedding_model.fit(x_train_data, _val_for_fit(y_train_data), 
                    validation_data=(x_valid_data,_val_for_fit(y_valid_data)),
                    epochs=nb_epoch, 
                    batch_size=256
                           # callbacks=[self.checkpointer],
                )

valid_rmspe=calc_predict_rmspe(embedding_model,x_valid_data,y_valid_data)
train_rmspe=calc_predict_rmspe(embedding_model,x_train_data,y_train_data)
print "train_rmspe :{:f},\tvalid_rmspe:{:f}".format(train_rmspe,valid_rmspe)

Train on 786180 samples, validate on 45852 samples
Epoch 1/1
786180/786180 [==============================] - 54s - loss: 0.2104 - val_loss: 0.1472
train_rmspe :0.156297,	valid_rmspe:0.149000


In [44]:
nb_epoch=2

embedding_model=create_embedding_model()
embedding_model.fit(x_train_data, _val_for_fit(y_train_data), 
                    validation_data=(x_valid_data,_val_for_fit(y_valid_data)),
                    epochs=nb_epoch, 
                    batch_size=256
                           # callbacks=[self.checkpointer],
                )

valid_rmspe=calc_predict_rmspe(embedding_model,x_valid_data,y_valid_data)
train_rmspe=calc_predict_rmspe(embedding_model,x_train_data,y_train_data)
print "train_rmspe :{:f},\tvalid_rmspe:{:f}".format(train_rmspe,valid_rmspe)

Train on 786180 samples, validate on 45852 samples
Epoch 1/2
786180/786180 [==============================] - 58s - loss: 0.1933 - val_loss: 0.1390
Epoch 2/2
786180/786180 [==============================] - 57s - loss: 0.1175 - val_loss: 0.1366
train_rmspe :0.130764,	valid_rmspe:0.138997


In [43]:
nb_epoch=3

embedding_model=create_embedding_model()
embedding_model.fit(x_train_data, _val_for_fit(y_train_data), 
                    validation_data=(x_valid_data,_val_for_fit(y_valid_data)),
                    epochs=nb_epoch, 
                    batch_size=256
                           # callbacks=[self.checkpointer],
                )

valid_rmspe=calc_predict_rmspe(embedding_model,x_valid_data,y_valid_data)
train_rmspe=calc_predict_rmspe(embedding_model,x_train_data,y_train_data)
print "train_rmspe :{:f},\tvalid_rmspe:{:f}".format(train_rmspe,valid_rmspe)

Train on 786180 samples, validate on 45852 samples
Epoch 1/3
786180/786180 [==============================] - 59s - loss: 0.2167 - val_loss: 0.1570
Epoch 2/3
786180/786180 [==============================] - 60s - loss: 0.1284 - val_loss: 0.1342
Epoch 3/3
786180/786180 [==============================] - 58s - loss: 0.1094 - val_loss: 0.1299
train_rmspe :0.106766,	valid_rmspe:0.132173


In [45]:
nb_epoch=4

embedding_model=create_embedding_model()
embedding_model.fit(x_train_data, _val_for_fit(y_train_data), 
                    validation_data=(x_valid_data,_val_for_fit(y_valid_data)),
                    epochs=nb_epoch, 
                    batch_size=256
                           # callbacks=[self.checkpointer],
                )

valid_rmspe=calc_predict_rmspe(embedding_model,x_valid_data,y_valid_data)
train_rmspe=calc_predict_rmspe(embedding_model,x_train_data,y_train_data)
print "train_rmspe :{:f},\tvalid_rmspe:{:f}".format(train_rmspe,valid_rmspe)

Train on 786180 samples, validate on 45852 samples
Epoch 1/4
786180/786180 [==============================] - 62s - loss: 0.1854 - val_loss: 0.1361
Epoch 2/4
786180/786180 [==============================] - 59s - loss: 0.1134 - val_loss: 0.1369
Epoch 3/4
786180/786180 [==============================] - 58s - loss: 0.1000 - val_loss: 0.1359
Epoch 4/4
786180/786180 [==============================] - 60s - loss: 0.0973 - val_loss: 0.1530
train_rmspe :0.120473,	valid_rmspe:0.155199


In [46]:
nb_epoch=5

embedding_model=create_embedding_model()
embedding_model.fit(x_train_data, _val_for_fit(y_train_data), 
                    validation_data=(x_valid_data,_val_for_fit(y_valid_data)),
                    epochs=nb_epoch, 
                    batch_size=256
                           # callbacks=[self.checkpointer],
                )

valid_rmspe=calc_predict_rmspe(embedding_model,x_valid_data,y_valid_data)
train_rmspe=calc_predict_rmspe(embedding_model,x_train_data,y_train_data)
print "train_rmspe :{:f},\tvalid_rmspe:{:f}".format(train_rmspe,valid_rmspe)

Train on 786180 samples, validate on 45852 samples
Epoch 1/5
786180/786180 [==============================] - 65s - loss: 0.1848 - val_loss: 0.1450
Epoch 2/5
786180/786180 [==============================] - 61s - loss: 0.1096 - val_loss: 0.1412
Epoch 3/5
786180/786180 [==============================] - 59s - loss: 0.1001 - val_loss: 0.1532
Epoch 4/5
786180/786180 [==============================] - 59s - loss: 0.0965 - val_loss: 0.1427
Epoch 5/5
786180/786180 [==============================] - 56s - loss: 0.0943 - val_loss: 0.1500
train_rmspe :0.112923,	valid_rmspe:0.152082


### 思路：重写loss函数
- 使用其他函数作为loss函数，

In [42]:
def calc_embedding_space(x):
    dict_map={
        1115:100,
        365:80,
        31:15,
        16:10,
        15:10,
        12:8,
        7:6,
        4:3,        
        3:2,
        2:1,
        1:1
    }
    return dict_map[x]


models=[]
for ii in feature_x_list:
    input_range=dict_feature_range[ii]
    
#     if input_range ==2 :
#         temp_name='Dense_'+ii
#         model= Sequential(name=temp_name)
#         model.add(Dense(1, input_dim=1))
#         models.append(model)
#     else:    
    embedding_space=calc_embedding_space(input_range)        
    temp_name='Embedding_'+ii
    model=Sequential(name=temp_name)
    model.add(Embedding(input_range,embedding_space,input_length=1))
    model.add(Reshape((embedding_space, ), input_shape=(1,embedding_space)))
    models.append(model)

    
max_log_y=np.max(np.log(y_train_data))

def _val_for_fit(val):
    val=np.log(val)
    return val/max_log_y

def _val_for_pred(val):
    return tf.exp(val*max_log_y)        

def loss_func_rmspe(y_true,y_pred):
    return tf.sqrt(tf.reduce_mean(tf.pow((_val_for_pred(y_pred)-_val_for_pred(y_true))/_val_for_pred(y_true),2.0)))

def calc_predict_rmspe(trained_model,x_data,y_data):
    predicted_value=trained_model.predict( x_data).flatten()
    return np.sqrt(np.mean(np.power((np.exp(predicted_value*max_log_y)-y_data.astype(np.float32))/y_data.astype(np.float32),2.0)))    
        
def create_embedding_model():
    dropout_rate=0.2

    embedding_model=Sequential()
    embedding_model.add(Merge(models,mode='concat',concat_axis=-1))

    embedding_model.add(Dense(1024,kernel_initializer='uniform'))
    embedding_model.add(Activation('relu'))
    embedding_model.add(Dense(256,kernel_initializer='uniform'))
    embedding_model.add(Activation('relu'))
    embedding_model.add(Dense(64,kernel_initializer='uniform'))
    embedding_model.add(Activation('relu'))
    embedding_model.add(Dropout(dropout_rate))
    embedding_model.add(Dense(1))
    embedding_model.add(Activation('sigmoid'))

    embedding_model.compile(loss=loss_func_rmspe,optimizer='adam')
    return embedding_model


In [40]:
nb_epoch=1

embedding_model=create_embedding_model()
embedding_model.fit(x_train_data, _val_for_fit(y_train_data), 
                    validation_data=(x_valid_data,_val_for_fit(y_valid_data)),
                    epochs=nb_epoch, 
                    batch_size=256
                           # callbacks=[self.checkpointer],
                )

valid_rmspe=calc_predict_rmspe(embedding_model,x_valid_data,y_valid_data)
train_rmspe=calc_predict_rmspe(embedding_model,x_train_data,y_train_data)
print "train_rmspe :{:f},\tvalid_rmspe:{:f}".format(train_rmspe,valid_rmspe)

Train on 786180 samples, validate on 45852 samples
Epoch 1/1
786180/786180 [==============================] - 54s - loss: 0.2104 - val_loss: 0.1472
train_rmspe :0.156297,	valid_rmspe:0.149000


In [ ]:
nb_epoch=2

embedding_model=create_embedding_model()
embedding_model.fit(x_train_data, _val_for_fit(y_train_data), 
                    validation_data=(x_valid_data,_val_for_fit(y_valid_data)),
                    epochs=nb_epoch, 
                    batch_size=256
                           # callbacks=[self.checkpointer],
                )

valid_rmspe=calc_predict_rmspe(embedding_model,x_valid_data,y_valid_data)
train_rmspe=calc_predict_rmspe(embedding_model,x_train_data,y_train_data)
print "train_rmspe :{:f},\tvalid_rmspe:{:f}".format(train_rmspe,valid_rmspe)

Train on 786180 samples, validate on 45852 samples
Epoch 1/2
786180/786180 [==============================] - 58s - loss: 0.1933 - val_loss: 0.1390
Epoch 2/2
786180/786180 [==============================] - 57s - loss: 0.1175 - val_loss: 0.1366
train_rmspe :0.130764,	valid_rmspe:0.138997


In [ ]:
nb_epoch=3

embedding_model=create_embedding_model()
embedding_model.fit(x_train_data, _val_for_fit(y_train_data), 
                    validation_data=(x_valid_data,_val_for_fit(y_valid_data)),
                    epochs=nb_epoch, 
                    batch_size=256
                           # callbacks=[self.checkpointer],
                )

valid_rmspe=calc_predict_rmspe(embedding_model,x_valid_data,y_valid_data)
train_rmspe=calc_predict_rmspe(embedding_model,x_train_data,y_train_data)
print "train_rmspe :{:f},\tvalid_rmspe:{:f}".format(train_rmspe,valid_rmspe)

Train on 786180 samples, validate on 45852 samples
Epoch 1/3
786180/786180 [==============================] - 59s - loss: 0.2167 - val_loss: 0.1570
Epoch 2/3
786180/786180 [==============================] - 60s - loss: 0.1284 - val_loss: 0.1342
Epoch 3/3
786180/786180 [==============================] - 58s - loss: 0.1094 - val_loss: 0.1299
train_rmspe :0.106766,	valid_rmspe:0.132173


In [ ]:
nb_epoch=4

embedding_model=create_embedding_model()
embedding_model.fit(x_train_data, _val_for_fit(y_train_data), 
                    validation_data=(x_valid_data,_val_for_fit(y_valid_data)),
                    epochs=nb_epoch, 
                    batch_size=256
                           # callbacks=[self.checkpointer],
                )

valid_rmspe=calc_predict_rmspe(embedding_model,x_valid_data,y_valid_data)
train_rmspe=calc_predict_rmspe(embedding_model,x_train_data,y_train_data)
print "train_rmspe :{:f},\tvalid_rmspe:{:f}".format(train_rmspe,valid_rmspe)

In [ ]:
nb_epoch=5

embedding_model=create_embedding_model()
embedding_model.fit(x_train_data, _val_for_fit(y_train_data), 
                    validation_data=(x_valid_data,_val_for_fit(y_valid_data)),
                    epochs=nb_epoch, 
                    batch_size=256
                           # callbacks=[self.checkpointer],
                )

valid_rmspe=calc_predict_rmspe(embedding_model,x_valid_data,y_valid_data)
train_rmspe=calc_predict_rmspe(embedding_model,x_train_data,y_train_data)
print "train_rmspe :{:f},\tvalid_rmspe:{:f}".format(train_rmspe,valid_rmspe)

### 思路：重写loss函数
- 使用其他函数作为loss函数，

In [28]:
def calc_embedding_space(x):
    dict_map={
        1115:50,
        365:30,
        31:15,
        16:10,
        15:10,
        12:8,
        7:6,
        4:3,        
        3:2,
        2:1,
        1:1
    }
    return dict_map[x]


models=[]
for ii in feature_x_list:
    input_range=dict_feature_range[ii]
    
#     if input_range ==2 :
#         temp_name='Dense_'+ii
#         model= Sequential(name=temp_name)
#         model.add(Dense(1, input_dim=1))
#         models.append(model)
#     else:    
    embedding_space=calc_embedding_space(input_range)        
    temp_name='Embedding_'+ii
    model=Sequential(name=temp_name)
    model.add(Embedding(input_range,embedding_space,input_length=1))
    model.add(Reshape((embedding_space, ), input_shape=(1,embedding_space)))
    models.append(model)

    
max_log_y=np.max(np.log(y_train_data))

def _val_for_fit(val):
    val=np.log(val)
    return val/max_log_y

def _val_for_pred(val):
    return tf.exp(val*max_log_y)        

def loss_func_rmspe(y_true,y_pred):
    return tf.sqrt(tf.reduce_mean(tf.pow((_val_for_pred(y_pred)-_val_for_pred(y_true))/_val_for_pred(y_true),2.0)))

def calc_predict_rmspe(trained_model,x_data,y_data):
    predicted_value=trained_model.predict( x_data).flatten()
    return np.sqrt(np.mean(np.power((np.exp(predicted_value*max_log_y)-y_data.astype(np.float32))/y_data.astype(np.float32),2.0)))    
        
def create_embedding_model():
    dropout_rate=0.1

    embedding_model=Sequential()
    embedding_model.add(Merge(models,mode='concat',concat_axis=-1))

    embedding_model.add(Dropout(dropout_rate))    
    embedding_model.add(Dense(512,
                              kernel_initializer='uniform',
                              kernel_regularizer=regularizers.l2(0.001),
#                               activity_regularizer=regularizers.l1(0.01)
                             ))
    embedding_model.add(Activation('relu'))
    embedding_model.add(Dense(256,
                              kernel_initializer='uniform',
                              kernel_regularizer=regularizers.l2(0.001),
#                               activity_regularizer=regularizers.l1(0.01)
                             ))
                             
    embedding_model.add(Activation('relu'))
    embedding_model.add(Dense(64,
                              kernel_initializer='uniform',
                            kernel_regularizer=regularizers.l2(0.001),
#                               activity_regularizer=regularizers.l1(0.01)
                             ))
                        
    embedding_model.add(Activation('relu'))

    embedding_model.add(Dense(1))
    embedding_model.add(Activation('sigmoid'))
    
    
    adam=keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
    embedding_model.compile(loss=loss_func_rmspe,optimizer=adam)
    return embedding_model


In [29]:
nb_epoch=1

embedding_model=create_embedding_model()
embedding_model.fit(x_train_data, _val_for_fit(y_train_data), 
                    validation_data=(x_valid_data,_val_for_fit(y_valid_data)),
                    epochs=nb_epoch, 
                    batch_size=256
                           # callbacks=[self.checkpointer],
                )

valid_rmspe=calc_predict_rmspe(embedding_model,x_valid_data,y_valid_data)
train_rmspe=calc_predict_rmspe(embedding_model,x_train_data,y_train_data)
print "train_rmspe :{:f},\tvalid_rmspe:{:f}".format(train_rmspe,valid_rmspe)

Train on 786180 samples, validate on 45852 samples
Epoch 1/1
786180/786180 [==============================] - 49s - loss: 0.1942 - val_loss: 0.1913
train_rmspe :0.152905,	valid_rmspe:0.180233


In [30]:
nb_epoch=2

embedding_model=create_embedding_model()
embedding_model.fit(x_train_data, _val_for_fit(y_train_data), 
                    validation_data=(x_valid_data,_val_for_fit(y_valid_data)),
                    epochs=nb_epoch, 
                    batch_size=256
                           # callbacks=[self.checkpointer],
                )

valid_rmspe=calc_predict_rmspe(embedding_model,x_valid_data,y_valid_data)
train_rmspe=calc_predict_rmspe(embedding_model,x_train_data,y_train_data)
print "train_rmspe :{:f},\tvalid_rmspe:{:f}".format(train_rmspe,valid_rmspe)

Train on 786180 samples, validate on 45852 samples
Epoch 1/2
786180/786180 [==============================] - 49s - loss: 0.1732 - val_loss: 0.1795
Epoch 2/2
786180/786180 [==============================] - 47s - loss: 0.1443 - val_loss: 0.2005
train_rmspe :0.169788,	valid_rmspe:0.193751


In [32]:
nb_epoch=3

embedding_model=create_embedding_model()
embedding_model.fit(x_train_data, _val_for_fit(y_train_data), 
                    validation_data=(x_valid_data,_val_for_fit(y_valid_data)),
                    epochs=nb_epoch, 
                    batch_size=256
                           # callbacks=[self.checkpointer],
                )

valid_rmspe=calc_predict_rmspe(embedding_model,x_valid_data,y_valid_data)
train_rmspe=calc_predict_rmspe(embedding_model,x_train_data,y_train_data)
print "train_rmspe :{:f},\tvalid_rmspe:{:f}".format(train_rmspe,valid_rmspe)

Train on 786180 samples, validate on 45852 samples
Epoch 1/3
786180/786180 [==============================] - 45s - loss: 0.1635 - val_loss: 0.2001
Epoch 2/3
786180/786180 [==============================] - 45s - loss: 0.1401 - val_loss: 0.1819
Epoch 3/3
786180/786180 [==============================] - 45s - loss: 0.1350 - val_loss: 0.1911
train_rmspe :0.139354,	valid_rmspe:0.183635


### 思路：重写loss函数
- 使用其他函数作为loss函数，

In [ ]:
def calc_embedding_space(x):
    dict_map={
        1115:50,
        365:30,
        31:15,
        16:10,
        15:10,
        12:8,
        7:6,
        4:3,        
        3:2,
        2:1,
        1:1
    }
    return dict_map[x]


models=[]
for ii in feature_x_list:
    input_range=dict_feature_range[ii]
    
#     if input_range ==2 :
#         temp_name='Dense_'+ii
#         model= Sequential(name=temp_name)
#         model.add(Dense(1, input_dim=1))
#         models.append(model)
#     else:    
    embedding_space=calc_embedding_space(input_range)        
    temp_name='Embedding_'+ii
    model=Sequential(name=temp_name)
    model.add(Embedding(input_range,embedding_space,input_length=1))
    model.add(Reshape((embedding_space, ), input_shape=(1,embedding_space)))
    models.append(model)

    
max_log_y=np.max(np.log(y_train_data))

def _val_for_fit(val):
    val=np.log(val)
    return val/max_log_y

def _val_for_pred(val):
    return tf.exp(val*max_log_y)        

def loss_func_rmspe(y_true,y_pred):
    return tf.sqrt(tf.reduce_mean(tf.pow((_val_for_pred(y_pred)-_val_for_pred(y_true))/_val_for_pred(y_true),2.0)))

def calc_predict_rmspe(trained_model,x_data,y_data):
    predicted_value=trained_model.predict( x_data).flatten()
    return np.sqrt(np.mean(np.power((np.exp(predicted_value*max_log_y)-y_data.astype(np.float32))/y_data.astype(np.float32),2.0)))    
        
def create_embedding_model():
    dropout_rate=0.0

    embedding_model=Sequential()
    embedding_model.add(Merge(models,mode='concat',concat_axis=-1))

    embedding_model.add(Dropout(dropout_rate))  
    
    embedding_model.add(Dense(512,
                              kernel_initializer='uniform',
#                               kernel_regularizer=regularizers.l2(0.001),
#                               activity_regularizer=regularizers.l1(0.01)
                             ))
    embedding_model.add(Activation('relu'))
    embedding_model.add(Dense(256,
                              kernel_initializer='uniform',
#                               kernel_regularizer=regularizers.l2(0.001),
#                               activity_regularizer=regularizers.l1(0.01)
                             ))
                             
    embedding_model.add(Activation('relu'))
    embedding_model.add(Dense(64,
                              kernel_initializer='uniform',
#                               kernel_regularizer=regularizers.l2(0.001),
#                               activity_regularizer=regularizers.l1(0.01)
                             ))
                        
    embedding_model.add(Activation('relu'))



    embedding_model.add(Dense(1))
    embedding_model.add(Activation('sigmoid'))
    
    
    adam=keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
    embedding_model.compile(loss=loss_func_rmspe,optimizer=adam)
    return embedding_model


In [16]:
nb_epoch=1

embedding_model=create_embedding_model()
embedding_model.fit(x_train_data, _val_for_fit(y_train_data), 
                    validation_data=(x_valid_data,_val_for_fit(y_valid_data)),
                    epochs=nb_epoch, 
                    batch_size=512
                           # callbacks=[self.checkpointer],
                )

valid_rmspe=calc_predict_rmspe(embedding_model,x_valid_data,y_valid_data)
train_rmspe=calc_predict_rmspe(embedding_model,x_train_data,y_train_data)
print "train_rmspe :{:f},\tvalid_rmspe:{:f}".format(train_rmspe,valid_rmspe)

Train on 786180 samples, validate on 45852 samples
Epoch 1/1
786180/786180 [==============================] - 27s - loss: 0.2300 - val_loss: 0.1603
train_rmspe :0.163457,	valid_rmspe:0.161508


In [17]:
nb_epoch=2

embedding_model=create_embedding_model()
embedding_model.fit(x_train_data, _val_for_fit(y_train_data), 
                    validation_data=(x_valid_data,_val_for_fit(y_valid_data)),
                    epochs=nb_epoch, 
                    batch_size=512
                           # callbacks=[self.checkpointer],
                )

valid_rmspe=calc_predict_rmspe(embedding_model,x_valid_data,y_valid_data)
train_rmspe=calc_predict_rmspe(embedding_model,x_train_data,y_train_data)
print "train_rmspe :{:f},\tvalid_rmspe:{:f}".format(train_rmspe,valid_rmspe)

Train on 786180 samples, validate on 45852 samples
Epoch 1/2
786180/786180 [==============================] - 28s - loss: 0.2255 - val_loss: 0.1531
Epoch 2/2
786180/786180 [==============================] - 30s - loss: 0.1639 - val_loss: 0.1420
train_rmspe :0.156099,	valid_rmspe:0.143310


In [18]:
nb_epoch=3

embedding_model=create_embedding_model()
embedding_model.fit(x_train_data, _val_for_fit(y_train_data), 
                    validation_data=(x_valid_data,_val_for_fit(y_valid_data)),
                    epochs=nb_epoch, 
                    batch_size=256
                           # callbacks=[self.checkpointer],
                )

valid_rmspe=calc_predict_rmspe(embedding_model,x_valid_data,y_valid_data)
train_rmspe=calc_predict_rmspe(embedding_model,x_train_data,y_train_data)
print "train_rmspe :{:f},\tvalid_rmspe:{:f}".format(train_rmspe,valid_rmspe)

Train on 786180 samples, validate on 45852 samples
Epoch 1/3
786180/786180 [==============================] - 38s - loss: 0.1886 - val_loss: 0.1484
Epoch 2/3
786180/786180 [==============================] - 37s - loss: 0.1255 - val_loss: 0.1418
Epoch 3/3
786180/786180 [==============================] - 37s - loss: 0.1121 - val_loss: 0.1494
train_rmspe :0.125123,	valid_rmspe:0.151603


In [19]:
def calc_embedding_space(x):
    dict_map={
        1115:50,
        365:30,
        31:15,
        16:10,
        15:10,
        12:8,
        7:6,
        4:3,        
        3:2,
        2:1,
        1:1
    }
    return dict_map[x]


models=[]
for ii in feature_x_list:
    input_range=dict_feature_range[ii]
    
#     if input_range ==2 :
#         temp_name='Dense_'+ii
#         model= Sequential(name=temp_name)
#         model.add(Dense(1, input_dim=1))
#         models.append(model)
#     else:    
    embedding_space=calc_embedding_space(input_range)        
    temp_name='Embedding_'+ii
    model=Sequential(name=temp_name)
    model.add(Embedding(input_range,embedding_space,input_length=1))
    model.add(Reshape((embedding_space, ), input_shape=(1,embedding_space)))
    models.append(model)

    
max_log_y=np.max(np.log(y_train_data))

def _val_for_fit(val):
    val=np.log(val)
    return val/max_log_y

def _val_for_pred(val):
    return tf.exp(val*max_log_y)        

def loss_func_rmspe(y_true,y_pred):
    return tf.sqrt(tf.reduce_mean(tf.pow((_val_for_pred(y_pred)-_val_for_pred(y_true))/_val_for_pred(y_true),2.0)))

def calc_predict_rmspe(trained_model,x_data,y_data):
    predicted_value=trained_model.predict( x_data).flatten()
    return np.sqrt(np.mean(np.power((np.exp(predicted_value*max_log_y)-y_data.astype(np.float32))/y_data.astype(np.float32),2.0)))    
        
def create_embedding_model():
    dropout_rate=0.01

    embedding_model=Sequential()
    embedding_model.add(Merge(models,mode='concat',concat_axis=-1))

    embedding_model.add(Dropout(dropout_rate))  
    
    embedding_model.add(Dense(512,
                              kernel_initializer='uniform',
#                               kernel_regularizer=regularizers.l2(0.001),
#                               activity_regularizer=regularizers.l1(0.01)
                             ))
    embedding_model.add(Activation('relu'))
    embedding_model.add(Dense(256,
                              kernel_initializer='uniform',
#                               kernel_regularizer=regularizers.l2(0.001),
#                               activity_regularizer=regularizers.l1(0.01)
                             ))
                             
    embedding_model.add(Activation('relu'))
    embedding_model.add(Dense(64,
                              kernel_initializer='uniform',
#                               kernel_regularizer=regularizers.l2(0.001),
#                               activity_regularizer=regularizers.l1(0.01)
                             ))
                        
    embedding_model.add(Activation('relu'))



    embedding_model.add(Dense(1))
    embedding_model.add(Activation('sigmoid'))
    
    
    adam=keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
    embedding_model.compile(loss=loss_func_rmspe,optimizer=adam)
    return embedding_model


In [20]:
nb_epoch=1

embedding_model=create_embedding_model()
embedding_model.fit(x_train_data, _val_for_fit(y_train_data), 
                    validation_data=(x_valid_data,_val_for_fit(y_valid_data)),
                    epochs=nb_epoch, 
                    batch_size=512
                           # callbacks=[self.checkpointer],
                )

valid_rmspe=calc_predict_rmspe(embedding_model,x_valid_data,y_valid_data)
train_rmspe=calc_predict_rmspe(embedding_model,x_train_data,y_train_data)
print "train_rmspe :{:f},\tvalid_rmspe:{:f}".format(train_rmspe,valid_rmspe)

nb_epoch=2

embedding_model=create_embedding_model()
embedding_model.fit(x_train_data, _val_for_fit(y_train_data), 
                    validation_data=(x_valid_data,_val_for_fit(y_valid_data)),
                    epochs=nb_epoch, 
                    batch_size=512
                           # callbacks=[self.checkpointer],
                )

valid_rmspe=calc_predict_rmspe(embedding_model,x_valid_data,y_valid_data)
train_rmspe=calc_predict_rmspe(embedding_model,x_train_data,y_train_data)
print "train_rmspe :{:f},\tvalid_rmspe:{:f}".format(train_rmspe,valid_rmspe)

nb_epoch=3

embedding_model=create_embedding_model()
embedding_model.fit(x_train_data, _val_for_fit(y_train_data), 
                    validation_data=(x_valid_data,_val_for_fit(y_valid_data)),
                    epochs=nb_epoch, 
                    batch_size=512
                           # callbacks=[self.checkpointer],
                )

valid_rmspe=calc_predict_rmspe(embedding_model,x_valid_data,y_valid_data)
train_rmspe=calc_predict_rmspe(embedding_model,x_train_data,y_train_data)
print "train_rmspe :{:f},\tvalid_rmspe:{:f}".format(train_rmspe,valid_rmspe)

Train on 786180 samples, validate on 45852 samples
Epoch 1/1
786180/786180 [==============================] - 28s - loss: 0.1676 - val_loss: 0.1597
train_rmspe :0.173919,	valid_rmspe:0.161094
Train on 786180 samples, validate on 45852 samples
Epoch 1/2
786180/786180 [==============================] - 29s - loss: 0.1446 - val_loss: 0.1564
Epoch 2/2
786180/786180 [==============================] - 28s - loss: 0.1180 - val_loss: 0.1576
train_rmspe :0.125046,	valid_rmspe:0.158702
Train on 786180 samples, validate on 45852 samples
Epoch 1/3
786180/786180 [==============================] - 29s - loss: 0.1337 - val_loss: 0.1655
Epoch 2/3
786180/786180 [==============================] - 29s - loss: 0.1111 - val_loss: 0.1605
Epoch 3/3
786180/786180 [==============================] - 29s - loss: 0.1038 - val_loss: 0.1597
train_rmspe :0.108715,	valid_rmspe:0.160889


In [22]:
def calc_embedding_space(x):
    dict_map={
        1115:50,
        365:30,
        31:15,
        16:10,
        15:10,
        12:8,
        7:6,
        4:3,        
        3:2,
        2:1,
        1:1
    }
    return dict_map[x]


models=[]
for ii in feature_x_list:
    input_range=dict_feature_range[ii]
    
#     if input_range ==2 :
#         temp_name='Dense_'+ii
#         model= Sequential(name=temp_name)
#         model.add(Dense(1, input_dim=1))
#         models.append(model)
#     else:    
    embedding_space=calc_embedding_space(input_range)        
    temp_name='Embedding_'+ii
    model=Sequential(name=temp_name)
    model.add(Embedding(input_range,embedding_space,input_length=1))
    model.add(Reshape((embedding_space, ), input_shape=(1,embedding_space)))
    models.append(model)

    
max_log_y=np.max(np.log(y_train_data))

def _val_for_fit(val):
    val=np.log(val)
    return val/max_log_y

def _val_for_pred(val):
    return tf.exp(val*max_log_y)        

def loss_func_rmspe(y_true,y_pred):
    return tf.sqrt(tf.reduce_mean(tf.pow((_val_for_pred(y_pred)-_val_for_pred(y_true))/_val_for_pred(y_true),2.0)))

def calc_predict_rmspe(trained_model,x_data,y_data):
    predicted_value=trained_model.predict( x_data).flatten()
    return np.sqrt(np.mean(np.power((np.exp(predicted_value*max_log_y)-y_data.astype(np.float32))/y_data.astype(np.float32),2.0)))    
        
def create_embedding_model():
    dropout_rate=0.05

    embedding_model=Sequential()
    embedding_model.add(Merge(models,mode='concat',concat_axis=-1))

    embedding_model.add(Dropout(dropout_rate))  
    
    embedding_model.add(Dense(512,
                              kernel_initializer='uniform',
#                               kernel_regularizer=regularizers.l2(0.001),
#                               activity_regularizer=regularizers.l1(0.01)
                             ))
    embedding_model.add(Activation('relu'))
    embedding_model.add(Dense(256,
                              kernel_initializer='uniform',
#                               kernel_regularizer=regularizers.l2(0.001),
#                               activity_regularizer=regularizers.l1(0.01)
                             ))
                             
    embedding_model.add(Activation('relu'))
    embedding_model.add(Dense(64,
                              kernel_initializer='uniform',
#                               kernel_regularizer=regularizers.l2(0.001),
#                               activity_regularizer=regularizers.l1(0.01)
                             ))
                        
    embedding_model.add(Activation('relu'))



    embedding_model.add(Dense(1))
    embedding_model.add(Activation('sigmoid'))
    
    
    adam=keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
    embedding_model.compile(loss=loss_func_rmspe,optimizer=adam)
    return embedding_model


In [23]:
nb_epoch=1

embedding_model=create_embedding_model()
embedding_model.fit(x_train_data, _val_for_fit(y_train_data), 
                    validation_data=(x_valid_data,_val_for_fit(y_valid_data)),
                    epochs=nb_epoch, 
                    batch_size=512
                           # callbacks=[self.checkpointer],
                )

valid_rmspe=calc_predict_rmspe(embedding_model,x_valid_data,y_valid_data)
train_rmspe=calc_predict_rmspe(embedding_model,x_train_data,y_train_data)
print "train_rmspe :{:f},\tvalid_rmspe:{:f}".format(train_rmspe,valid_rmspe)

nb_epoch=2

embedding_model=create_embedding_model()
embedding_model.fit(x_train_data, _val_for_fit(y_train_data), 
                    validation_data=(x_valid_data,_val_for_fit(y_valid_data)),
                    epochs=nb_epoch, 
                    batch_size=512
                           # callbacks=[self.checkpointer],
                )

valid_rmspe=calc_predict_rmspe(embedding_model,x_valid_data,y_valid_data)
train_rmspe=calc_predict_rmspe(embedding_model,x_train_data,y_train_data)
print "train_rmspe :{:f},\tvalid_rmspe:{:f}".format(train_rmspe,valid_rmspe)

nb_epoch=3

embedding_model=create_embedding_model()
embedding_model.fit(x_train_data, _val_for_fit(y_train_data), 
                    validation_data=(x_valid_data,_val_for_fit(y_valid_data)),
                    epochs=nb_epoch, 
                    batch_size=512
                           # callbacks=[self.checkpointer],
                )

valid_rmspe=calc_predict_rmspe(embedding_model,x_valid_data,y_valid_data)
train_rmspe=calc_predict_rmspe(embedding_model,x_train_data,y_train_data)
print "train_rmspe :{:f},\tvalid_rmspe:{:f}".format(train_rmspe,valid_rmspe)

Train on 786180 samples, validate on 45852 samples
Epoch 1/1
786180/786180 [==============================] - 31s - loss: 0.1772 - val_loss: 0.1608
train_rmspe :0.162839,	valid_rmspe:0.162016
Train on 786180 samples, validate on 45852 samples
Epoch 1/2
786180/786180 [==============================] - 30s - loss: 0.1491 - val_loss: 0.1691
Epoch 2/2
786180/786180 [==============================] - 29s - loss: 0.1256 - val_loss: 0.1570
train_rmspe :0.143385,	valid_rmspe:0.157924
Train on 786180 samples, validate on 45852 samples
Epoch 1/3
786180/786180 [==============================] - 30s - loss: 0.1439 - val_loss: 0.1650
Epoch 2/3
786180/786180 [==============================] - 29s - loss: 0.1218 - val_loss: 0.1546
Epoch 3/3
786180/786180 [==============================] - 29s - loss: 0.1114 - val_loss: 0.1492
train_rmspe :0.122826,	valid_rmspe:0.150442


In [24]:
def calc_embedding_space(x):
    dict_map={
        1115:50,
        365:30,
        31:15,
        16:10,
        15:10,
        12:8,
        7:6,
        4:3,        
        3:2,
        2:1,
        1:1
    }
    return dict_map[x]


models=[]
for ii in feature_x_list:
    input_range=dict_feature_range[ii]
    
#     if input_range ==2 :
#         temp_name='Dense_'+ii
#         model= Sequential(name=temp_name)
#         model.add(Dense(1, input_dim=1))
#         models.append(model)
#     else:    
    embedding_space=calc_embedding_space(input_range)        
    temp_name='Embedding_'+ii
    model=Sequential(name=temp_name)
    model.add(Embedding(input_range,embedding_space,input_length=1))
    model.add(Reshape((embedding_space, ), input_shape=(1,embedding_space)))
    models.append(model)

    
max_log_y=np.max(np.log(y_train_data))

def _val_for_fit(val):
    val=np.log(val)
    return val/max_log_y

def _val_for_pred(val):
    return tf.exp(val*max_log_y)        

def loss_func_rmspe(y_true,y_pred):
    return tf.sqrt(tf.reduce_mean(tf.pow((_val_for_pred(y_pred)-_val_for_pred(y_true))/_val_for_pred(y_true),2.0)))

def calc_predict_rmspe(trained_model,x_data,y_data):
    predicted_value=trained_model.predict( x_data).flatten()
    return np.sqrt(np.mean(np.power((np.exp(predicted_value*max_log_y)-y_data.astype(np.float32))/y_data.astype(np.float32),2.0)))    
        
def create_embedding_model():
    dropout_rate=0.02

    embedding_model=Sequential()
    embedding_model.add(Merge(models,mode='concat',concat_axis=-1))

    embedding_model.add(Dropout(dropout_rate))  
    
    embedding_model.add(Dense(512,
                              kernel_initializer='uniform',
#                               kernel_regularizer=regularizers.l2(0.001),
#                               activity_regularizer=regularizers.l1(0.01)
                             ))
    embedding_model.add(Activation('relu'))
    embedding_model.add(Dense(256,
                              kernel_initializer='uniform',
#                               kernel_regularizer=regularizers.l2(0.001),
#                               activity_regularizer=regularizers.l1(0.01)
                             ))
                             
    embedding_model.add(Activation('relu'))
    embedding_model.add(Dense(64,
                              kernel_initializer='uniform',
#                               kernel_regularizer=regularizers.l2(0.001),
#                               activity_regularizer=regularizers.l1(0.01)
                             ))
                        
    embedding_model.add(Activation('relu'))



    embedding_model.add(Dense(1))
    embedding_model.add(Activation('sigmoid'))
    
    
    adam=keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
    embedding_model.compile(loss=loss_func_rmspe,optimizer=adam)
    return embedding_model


In [25]:
nb_epoch=1

embedding_model=create_embedding_model()
embedding_model.fit(x_train_data, _val_for_fit(y_train_data), 
                    validation_data=(x_valid_data,_val_for_fit(y_valid_data)),
                    epochs=nb_epoch, 
                    batch_size=512
                           # callbacks=[self.checkpointer],
                )

valid_rmspe=calc_predict_rmspe(embedding_model,x_valid_data,y_valid_data)
train_rmspe=calc_predict_rmspe(embedding_model,x_train_data,y_train_data)
print "train_rmspe :{:f},\tvalid_rmspe:{:f}".format(train_rmspe,valid_rmspe)

nb_epoch=2

embedding_model=create_embedding_model()
embedding_model.fit(x_train_data, _val_for_fit(y_train_data), 
                    validation_data=(x_valid_data,_val_for_fit(y_valid_data)),
                    epochs=nb_epoch, 
                    batch_size=512
                           # callbacks=[self.checkpointer],
                )

valid_rmspe=calc_predict_rmspe(embedding_model,x_valid_data,y_valid_data)
train_rmspe=calc_predict_rmspe(embedding_model,x_train_data,y_train_data)
print "train_rmspe :{:f},\tvalid_rmspe:{:f}".format(train_rmspe,valid_rmspe)

nb_epoch=3

embedding_model=create_embedding_model()
embedding_model.fit(x_train_data, _val_for_fit(y_train_data), 
                    validation_data=(x_valid_data,_val_for_fit(y_valid_data)),
                    epochs=nb_epoch, 
                    batch_size=512
                           # callbacks=[self.checkpointer],
                )

valid_rmspe=calc_predict_rmspe(embedding_model,x_valid_data,y_valid_data)
train_rmspe=calc_predict_rmspe(embedding_model,x_train_data,y_train_data)
print "train_rmspe :{:f},\tvalid_rmspe:{:f}".format(train_rmspe,valid_rmspe)

Train on 786180 samples, validate on 45852 samples
Epoch 1/1
786180/786180 [==============================] - 29s - loss: 0.1708 - val_loss: 0.1548
train_rmspe :0.162046,	valid_rmspe:0.156194
Train on 786180 samples, validate on 45852 samples
Epoch 1/2
786180/786180 [==============================] - 29s - loss: 0.1473 - val_loss: 0.1586
Epoch 2/2
786180/786180 [==============================] - 29s - loss: 0.1197 - val_loss: 0.1556
train_rmspe :0.156010,	valid_rmspe:0.156525
Train on 786180 samples, validate on 45852 samples
Epoch 1/3
786180/786180 [==============================] - 32s - loss: 0.1350 - val_loss: 0.1648
Epoch 2/3
786180/786180 [==============================] - 30s - loss: 0.1149 - val_loss: 0.1565
Epoch 3/3
786180/786180 [==============================] - 30s - loss: 0.1079 - val_loss: 0.1573
train_rmspe :0.118726,	valid_rmspe:0.158317


In [26]:
def calc_embedding_space(x):
    dict_map={
        1115:100,
        365:80,
        31:15,
        16:10,
        15:10,
        12:8,
        7:6,
        4:3,        
        3:2,
        2:1,
        1:1
    }
    return dict_map[x]


models=[]
for ii in feature_x_list:
    input_range=dict_feature_range[ii]
    
#     if input_range ==2 :
#         temp_name='Dense_'+ii
#         model= Sequential(name=temp_name)
#         model.add(Dense(1, input_dim=1))
#         models.append(model)
#     else:    
    embedding_space=calc_embedding_space(input_range)        
    temp_name='Embedding_'+ii
    model=Sequential(name=temp_name)
    model.add(Embedding(input_range,embedding_space,input_length=1))
    model.add(Reshape((embedding_space, ), input_shape=(1,embedding_space)))
    models.append(model)

    
max_log_y=np.max(np.log(y_train_data))

def _val_for_fit(val):
    val=np.log(val)
    return val/max_log_y

def _val_for_pred(val):
    return tf.exp(val*max_log_y)        

def loss_func_rmspe(y_true,y_pred):
    return tf.sqrt(tf.reduce_mean(tf.pow((_val_for_pred(y_pred)-_val_for_pred(y_true))/_val_for_pred(y_true),2.0)))

def calc_predict_rmspe(trained_model,x_data,y_data):
    predicted_value=trained_model.predict( x_data).flatten()
    return np.sqrt(np.mean(np.power((np.exp(predicted_value*max_log_y)-y_data.astype(np.float32))/y_data.astype(np.float32),2.0)))    
        
def create_embedding_model():
    dropout_rate=0.25

    embedding_model=Sequential()
    embedding_model.add(Merge(models,mode='concat',concat_axis=-1))

    embedding_model.add(Dense(1024,kernel_initializer='uniform'))
    embedding_model.add(Activation('relu'))
    embedding_model.add(Dense(256,kernel_initializer='uniform'))
    embedding_model.add(Activation('relu'))
    embedding_model.add(Dense(64,kernel_initializer='uniform'))
    embedding_model.add(Activation('relu'))
    embedding_model.add(Dropout(dropout_rate))
    embedding_model.add(Dense(1))
    embedding_model.add(Activation('sigmoid'))

    embedding_model.compile(loss=loss_func_rmspe,optimizer='adam')
    return embedding_model


In [27]:
nb_epoch=1

embedding_model=create_embedding_model()
embedding_model.fit(x_train_data, _val_for_fit(y_train_data), 
                    validation_data=(x_valid_data,_val_for_fit(y_valid_data)),
                    epochs=nb_epoch, 
                    batch_size=512
                           # callbacks=[self.checkpointer],
                )

valid_rmspe=calc_predict_rmspe(embedding_model,x_valid_data,y_valid_data)
train_rmspe=calc_predict_rmspe(embedding_model,x_train_data,y_train_data)
print "train_rmspe :{:f},\tvalid_rmspe:{:f}".format(train_rmspe,valid_rmspe)

nb_epoch=2

embedding_model=create_embedding_model()
embedding_model.fit(x_train_data, _val_for_fit(y_train_data), 
                    validation_data=(x_valid_data,_val_for_fit(y_valid_data)),
                    epochs=nb_epoch, 
                    batch_size=512
                           # callbacks=[self.checkpointer],
                )

valid_rmspe=calc_predict_rmspe(embedding_model,x_valid_data,y_valid_data)
train_rmspe=calc_predict_rmspe(embedding_model,x_train_data,y_train_data)
print "train_rmspe :{:f},\tvalid_rmspe:{:f}".format(train_rmspe,valid_rmspe)

nb_epoch=3

embedding_model=create_embedding_model()
embedding_model.fit(x_train_data, _val_for_fit(y_train_data), 
                    validation_data=(x_valid_data,_val_for_fit(y_valid_data)),
                    epochs=nb_epoch, 
                    batch_size=512
                           # callbacks=[self.checkpointer],
                )

valid_rmspe=calc_predict_rmspe(embedding_model,x_valid_data,y_valid_data)
train_rmspe=calc_predict_rmspe(embedding_model,x_train_data,y_train_data)
print "train_rmspe :{:f},\tvalid_rmspe:{:f}".format(train_rmspe,valid_rmspe)

Train on 786180 samples, validate on 45852 samples
Epoch 1/1
786180/786180 [==============================] - 56s - loss: 0.2533 - val_loss: 0.1524
train_rmspe :0.173747,	valid_rmspe:0.153495
Train on 786180 samples, validate on 45852 samples
Epoch 1/2
786180/786180 [==============================] - 73s - loss: 0.2296 - val_loss: 0.1421
Epoch 2/2
786180/786180 [==============================] - 53s - loss: 0.1583 - val_loss: 0.1337
train_rmspe :0.168907,	valid_rmspe:0.135155
Train on 786180 samples, validate on 45852 samples
Epoch 1/3
786180/786180 [==============================] - 51s - loss: 0.2395 - val_loss: 0.1415
Epoch 2/3
786180/786180 [==============================] - 50s - loss: 0.1587 - val_loss: 0.1330
Epoch 3/3
786180/786180 [==============================] - 51s - loss: 0.1188 - val_loss: 0.1408
train_rmspe :0.114575,	valid_rmspe:0.142446


In [34]:
def calc_embedding_space(x):
    dict_map={
        1115:100,
        365:80,
        31:15,
        16:10,
        15:10,
        12:8,
        7:6,
        4:3,        
        3:2,
        2:1,
        1:1
    }
    return dict_map[x]


models=[]
for ii in feature_x_list:
    input_range=dict_feature_range[ii]
    
#     if input_range ==2 :
#         temp_name='Dense_'+ii
#         model= Sequential(name=temp_name)
#         model.add(Dense(1, input_dim=1))
#         models.append(model)
#     else:    
    embedding_space=calc_embedding_space(input_range)        
    temp_name='Embedding_'+ii
    model=Sequential(name=temp_name)
    model.add(Embedding(input_range,embedding_space,input_length=1))
    model.add(Reshape((embedding_space, ), input_shape=(1,embedding_space)))
    models.append(model)

    
max_log_y=np.max(np.log(y_train_data))

def _val_for_fit(val):
    val=np.log(val)
    return val/max_log_y

def _val_for_pred(val):
    return tf.exp(val*max_log_y)        

def loss_func_rmspe(y_true,y_pred):
    return tf.sqrt(tf.reduce_mean(tf.pow((_val_for_pred(y_pred)-_val_for_pred(y_true))/_val_for_pred(y_true),2.0)))

def calc_predict_rmspe(trained_model,x_data,y_data):
    predicted_value=trained_model.predict( x_data).flatten()
    return np.sqrt(np.mean(np.power((np.exp(predicted_value*max_log_y)-y_data.astype(np.float32))/y_data.astype(np.float32),2.0)))    
        
def create_embedding_model():
    dropout_rate=0.3

    embedding_model=Sequential()
    embedding_model.add(Merge(models,mode='concat',concat_axis=-1))

    embedding_model.add(Dense(1024,kernel_initializer='uniform'))
    embedding_model.add(Activation('relu'))
    embedding_model.add(Dense(256,kernel_initializer='uniform'))
    embedding_model.add(Activation('relu'))
    embedding_model.add(Dense(64,kernel_initializer='uniform'))
    embedding_model.add(Activation('relu'))
    embedding_model.add(Dropout(dropout_rate))
    embedding_model.add(Dense(1))
    embedding_model.add(Activation('sigmoid'))

    embedding_model.compile(loss=loss_func_rmspe,optimizer='adam')
    return embedding_model


In [35]:
nb_epoch=1

embedding_model=create_embedding_model()
embedding_model.fit(x_train_data, _val_for_fit(y_train_data), 
                    validation_data=(x_valid_data,_val_for_fit(y_valid_data)),
                    epochs=nb_epoch, 
                    batch_size=512
                           # callbacks=[self.checkpointer],
                )

valid_rmspe=calc_predict_rmspe(embedding_model,x_valid_data,y_valid_data)
train_rmspe=calc_predict_rmspe(embedding_model,x_train_data,y_train_data)
print "train_rmspe :{:f},\tvalid_rmspe:{:f}".format(train_rmspe,valid_rmspe)


Train on 786180 samples, validate on 45852 samples
Epoch 1/1
786180/786180 [==============================] - 57s - loss: 0.2742 - val_loss: 0.1530


KeyboardInterrupt: 

In [28]:
nb_epoch=1

embedding_model=create_embedding_model()
embedding_model.fit(x_train_data, _val_for_fit(y_train_data), 
                    validation_data=(x_valid_data,_val_for_fit(y_valid_data)),
                    epochs=nb_epoch, 
                    batch_size=512
                           # callbacks=[self.checkpointer],
                )

valid_rmspe=calc_predict_rmspe(embedding_model,x_valid_data,y_valid_data)
train_rmspe=calc_predict_rmspe(embedding_model,x_train_data,y_train_data)
print "train_rmspe :{:f},\tvalid_rmspe:{:f}".format(train_rmspe,valid_rmspe)

nb_epoch=2

embedding_model=create_embedding_model()
embedding_model.fit(x_train_data, _val_for_fit(y_train_data), 
                    validation_data=(x_valid_data,_val_for_fit(y_valid_data)),
                    epochs=nb_epoch, 
                    batch_size=512
                           # callbacks=[self.checkpointer],
                )

valid_rmspe=calc_predict_rmspe(embedding_model,x_valid_data,y_valid_data)
train_rmspe=calc_predict_rmspe(embedding_model,x_train_data,y_train_data)
print "train_rmspe :{:f},\tvalid_rmspe:{:f}".format(train_rmspe,valid_rmspe)

nb_epoch=3

embedding_model=create_embedding_model()
embedding_model.fit(x_train_data, _val_for_fit(y_train_data), 
                    validation_data=(x_valid_data,_val_for_fit(y_valid_data)),
                    epochs=nb_epoch, 
                    batch_size=512
                           # callbacks=[self.checkpointer],
                )

valid_rmspe=calc_predict_rmspe(embedding_model,x_valid_data,y_valid_data)
train_rmspe=calc_predict_rmspe(embedding_model,x_train_data,y_train_data)
print "train_rmspe :{:f},\tvalid_rmspe:{:f}".format(train_rmspe,valid_rmspe)

Train on 786180 samples, validate on 45852 samples
Epoch 1/1
786180/786180 [==============================] - 52s - loss: 0.2451 - val_loss: 0.1375
train_rmspe :0.136074,	valid_rmspe:0.138959
Train on 786180 samples, validate on 45852 samples
Epoch 1/2
786180/786180 [==============================] - 52s - loss: 0.2245 - val_loss: 0.5963
Epoch 2/2
786180/786180 [==============================] - 51s - loss: 0.1557 - val_loss: 0.1344
train_rmspe :0.148553,	valid_rmspe:0.135777
Train on 786180 samples, validate on 45852 samples
Epoch 1/3
786180/786180 [==============================] - 52s - loss: 0.2474 - val_loss: 0.1382
Epoch 2/3
786180/786180 [==============================] - 50s - loss: 0.1642 - val_loss: 0.1357
Epoch 3/3
786180/786180 [==============================] - 51s - loss: 0.1169 - val_loss: 0.1307
train_rmspe :0.146995,	valid_rmspe:0.132269


In [29]:
nb_epoch=4

embedding_model=create_embedding_model()
embedding_model.fit(x_train_data, _val_for_fit(y_train_data), 
                    validation_data=(x_valid_data,_val_for_fit(y_valid_data)),
                    epochs=nb_epoch, 
                    batch_size=512
                           # callbacks=[self.checkpointer],
                )

valid_rmspe=calc_predict_rmspe(embedding_model,x_valid_data,y_valid_data)
train_rmspe=calc_predict_rmspe(embedding_model,x_train_data,y_train_data)
print "train_rmspe :{:f},\tvalid_rmspe:{:f}".format(train_rmspe,valid_rmspe)
nb_epoch=5

embedding_model=create_embedding_model()
embedding_model.fit(x_train_data, _val_for_fit(y_train_data), 
                    validation_data=(x_valid_data,_val_for_fit(y_valid_data)),
                    epochs=nb_epoch, 
                    batch_size=512
                           # callbacks=[self.checkpointer],
                )

valid_rmspe=calc_predict_rmspe(embedding_model,x_valid_data,y_valid_data)
train_rmspe=calc_predict_rmspe(embedding_model,x_train_data,y_train_data)
print "train_rmspe :{:f},\tvalid_rmspe:{:f}".format(train_rmspe,valid_rmspe)

Train on 786180 samples, validate on 45852 samples
Epoch 1/4
786180/786180 [==============================] - 53s - loss: 0.2177 - val_loss: 0.1355
Epoch 2/4
786180/786180 [==============================] - 54s - loss: 0.1539 - val_loss: 0.1386
Epoch 3/4
786180/786180 [==============================] - 54s - loss: 0.1139 - val_loss: 0.1292
Epoch 4/4
786180/786180 [==============================] - 54s - loss: 0.1041 - val_loss: 0.1324
train_rmspe :0.140546,	valid_rmspe:0.133977
Train on 786180 samples, validate on 45852 samples
Epoch 1/5
786180/786180 [==============================] - 57s - loss: 0.2285 - val_loss: 0.3411
Epoch 2/5
786180/786180 [==============================] - 55s - loss: 0.1433 - val_loss: 0.1335
Epoch 3/5
786180/786180 [==============================] - 55s - loss: 0.1078 - val_loss: 0.1279
Epoch 4/5
786180/786180 [==============================] - 56s - loss: 0.1018 - val_loss: 0.1301
Epoch 5/5
786180/786180 [==============================] - 56s - loss: 0.0998 

In [30]:
def calc_embedding_space(x):
    dict_map={
        1115:100,
        365:80,
        31:15,
        16:10,
        15:10,
        12:8,
        7:6,
        4:3,        
        3:2,
        2:1,
        1:1
    }
    return dict_map[x]


models=[]
for ii in feature_x_list:
    input_range=dict_feature_range[ii]
    
#     if input_range ==2 :
#         temp_name='Dense_'+ii
#         model= Sequential(name=temp_name)
#         model.add(Dense(1, input_dim=1))
#         models.append(model)
#     else:    
    embedding_space=calc_embedding_space(input_range)        
    temp_name='Embedding_'+ii
    model=Sequential(name=temp_name)
    model.add(Embedding(input_range,embedding_space,input_length=1))
    model.add(Reshape((embedding_space, ), input_shape=(1,embedding_space)))
    models.append(model)

    
max_log_y=np.max(np.log(y_train_data))

def _val_for_fit(val):
    val=np.log(val)
    return val/max_log_y

def _val_for_pred(val):
    return tf.exp(val*max_log_y)        

def loss_func_rmspe(y_true,y_pred):
    return tf.sqrt(tf.reduce_mean(tf.pow((_val_for_pred(y_pred)-_val_for_pred(y_true))/_val_for_pred(y_true),2.0)))

def calc_predict_rmspe(trained_model,x_data,y_data):
    predicted_value=trained_model.predict( x_data).flatten()
    return np.sqrt(np.mean(np.power((np.exp(predicted_value*max_log_y)-y_data.astype(np.float32))/y_data.astype(np.float32),2.0)))    
        
def create_embedding_model():
    dropout_rate=0.35

    embedding_model=Sequential()
    embedding_model.add(Merge(models,mode='concat',concat_axis=-1))

    embedding_model.add(Dense(1024,kernel_initializer='uniform'))
    embedding_model.add(Activation('relu'))
    embedding_model.add(Dense(256,kernel_initializer='uniform'))
    embedding_model.add(Activation('relu'))
    embedding_model.add(Dense(64,kernel_initializer='uniform'))
    embedding_model.add(Activation('relu'))
    embedding_model.add(Dropout(dropout_rate))
    embedding_model.add(Dense(1))
    embedding_model.add(Activation('sigmoid'))

    embedding_model.compile(loss=loss_func_rmspe,optimizer='adam')
    return embedding_model


In [31]:
nb_epoch=1

embedding_model=create_embedding_model()
embedding_model.fit(x_train_data, _val_for_fit(y_train_data), 
                    validation_data=(x_valid_data,_val_for_fit(y_valid_data)),
                    epochs=nb_epoch, 
                    batch_size=512
                           # callbacks=[self.checkpointer],
                )

valid_rmspe=calc_predict_rmspe(embedding_model,x_valid_data,y_valid_data)
train_rmspe=calc_predict_rmspe(embedding_model,x_train_data,y_train_data)
print "train_rmspe :{:f},\tvalid_rmspe:{:f}".format(train_rmspe,valid_rmspe)

nb_epoch=2

embedding_model=create_embedding_model()
embedding_model.fit(x_train_data, _val_for_fit(y_train_data), 
                    validation_data=(x_valid_data,_val_for_fit(y_valid_data)),
                    epochs=nb_epoch, 
                    batch_size=512
                           # callbacks=[self.checkpointer],
                )

valid_rmspe=calc_predict_rmspe(embedding_model,x_valid_data,y_valid_data)
train_rmspe=calc_predict_rmspe(embedding_model,x_train_data,y_train_data)
print "train_rmspe :{:f},\tvalid_rmspe:{:f}".format(train_rmspe,valid_rmspe)

nb_epoch=3

embedding_model=create_embedding_model()
embedding_model.fit(x_train_data, _val_for_fit(y_train_data), 
                    validation_data=(x_valid_data,_val_for_fit(y_valid_data)),
                    epochs=nb_epoch, 
                    batch_size=512
                           # callbacks=[self.checkpointer],
                )

valid_rmspe=calc_predict_rmspe(embedding_model,x_valid_data,y_valid_data)
train_rmspe=calc_predict_rmspe(embedding_model,x_train_data,y_train_data)
print "train_rmspe :{:f},\tvalid_rmspe:{:f}".format(train_rmspe,valid_rmspe)

Train on 786180 samples, validate on 45852 samples
Epoch 1/1
786180/786180 [==============================] - 55s - loss: 0.2831 - val_loss: 0.1549
train_rmspe :0.182888,	valid_rmspe:0.156200
Train on 786180 samples, validate on 45852 samples
Epoch 1/2
786180/786180 [==============================] - 58s - loss: 0.2757 - val_loss: 0.1532
Epoch 2/2
786180/786180 [==============================] - 54s - loss: 0.1635 - val_loss: 0.1403
train_rmspe :0.140503,	valid_rmspe:0.141535
Train on 786180 samples, validate on 45852 samples
Epoch 1/3
786180/786180 [==============================] - 57s - loss: 0.2640 - val_loss: 0.1440
Epoch 2/3
786180/786180 [==============================] - 56s - loss: 0.1489 - val_loss: 0.1389
Epoch 3/3
786180/786180 [==============================] - 55s - loss: 0.1204 - val_loss: 0.1338
train_rmspe :0.131888,	valid_rmspe:0.135204


In [32]:
nb_epoch=4

embedding_model=create_embedding_model()
embedding_model.fit(x_train_data, _val_for_fit(y_train_data), 
                    validation_data=(x_valid_data,_val_for_fit(y_valid_data)),
                    epochs=nb_epoch, 
                    batch_size=512
                           # callbacks=[self.checkpointer],
                )

valid_rmspe=calc_predict_rmspe(embedding_model,x_valid_data,y_valid_data)
train_rmspe=calc_predict_rmspe(embedding_model,x_train_data,y_train_data)
print "train_rmspe :{:f},\tvalid_rmspe:{:f}".format(train_rmspe,valid_rmspe)

nb_epoch=5

embedding_model=create_embedding_model()
embedding_model.fit(x_train_data, _val_for_fit(y_train_data), 
                    validation_data=(x_valid_data,_val_for_fit(y_valid_data)),
                    epochs=nb_epoch, 
                    batch_size=512
                           # callbacks=[self.checkpointer],
                )

valid_rmspe=calc_predict_rmspe(embedding_model,x_valid_data,y_valid_data)
train_rmspe=calc_predict_rmspe(embedding_model,x_train_data,y_train_data)
print "train_rmspe :{:f},\tvalid_rmspe:{:f}".format(train_rmspe,valid_rmspe)

Train on 786180 samples, validate on 45852 samples
Epoch 1/4
786180/786180 [==============================] - 57s - loss: 0.2485 - val_loss: 0.1406
Epoch 2/4
786180/786180 [==============================] - 57s - loss: 0.1434 - val_loss: 0.1351
Epoch 3/4
786180/786180 [==============================] - 54s - loss: 0.1160 - val_loss: 0.1321
Epoch 4/4
786180/786180 [==============================] - 52s - loss: 0.1122 - val_loss: 0.1395
train_rmspe :0.096448,	valid_rmspe:0.141115
Train on 786180 samples, validate on 45852 samples
Epoch 1/5
786180/786180 [==============================] - 58s - loss: 0.2471 - val_loss: 0.1398
Epoch 2/5
786180/786180 [==============================] - 55s - loss: 0.1508 - val_loss: 0.1319
Epoch 3/5
786180/786180 [==============================] - 55s - loss: 0.1181 - val_loss: 0.1405
Epoch 4/5
786180/786180 [==============================] - 55s - loss: 0.1145 - val_loss: 0.1309
Epoch 5/5
173568/786180 [=====>........................] - ETA: 42s - loss: 0.

ValueError: I/O operation on closed file

In [44]:
def calc_embedding_space(x):
    dict_map={
        1115:100,
        365:80,
        31:15,
        16:10,
        15:10,
        12:8,
        7:6,
        4:3,        
        3:2,
        2:1,
        1:1
    }
    return dict_map[x]


models=[]
for ii in feature_x_list:
    input_range=dict_feature_range[ii]
    
#     if input_range ==2 :
#         temp_name='Dense_'+ii
#         model= Sequential(name=temp_name)
#         model.add(Dense(1, input_dim=1))
#         models.append(model)
#     else:    
    embedding_space=calc_embedding_space(input_range)        
    temp_name='Embedding_'+ii
    model=Sequential(name=temp_name)
    model.add(Embedding(input_range,embedding_space,input_length=1))
    model.add(Reshape((embedding_space, ), input_shape=(1,embedding_space)))
    models.append(model)

    
max_log_y=np.max(np.log(y_train_data))

def _val_for_fit(val):
    val=np.log(val)
    return val/max_log_y

def _val_for_pred(val):
    return tf.exp(val*max_log_y)        

def loss_func_rmspe(y_true,y_pred):
    return tf.sqrt(tf.reduce_mean(tf.pow((_val_for_pred(y_pred)-_val_for_pred(y_true))/_val_for_pred(y_true),2.0)))

def calc_predict_rmspe(trained_model,x_data,y_data):
    predicted_value=trained_model.predict( x_data).flatten()
    return np.sqrt(np.mean(np.power((np.exp(predicted_value*max_log_y)-y_data.astype(np.float32))/y_data.astype(np.float32),2.0)))    
        
def create_embedding_model():
    dropout_rate=0.3

    embedding_model=Sequential()
    embedding_model.add(Merge(models,mode='concat',concat_axis=-1))
    
    uniform=keras.initializers.RandomUniform(minval=-0.05, maxval=0.05, seed=seed_1)

    embedding_model.add(Dense(50,kernel_initializer=uniform))
    embedding_model.add(Activation('relu'))
#     embedding_model.add(Dense(256,kernel_initializer='uniform'))
#     embedding_model.add(Activation('relu'))
#     embedding_model.add(Dense(64,kernel_initializer='uniform'))
#     embedding_model.add(Activation('relu'))
    embedding_model.add(Dropout(dropout_rate,seed=seed_1))
    embedding_model.add(Dense(1))
    embedding_model.add(Activation('sigmoid'))

    embedding_model.compile(loss=loss_func_rmspe,optimizer='adam')
    return embedding_model


In [45]:
nb_epoch=1

embedding_model=create_embedding_model()
embedding_model.fit(x_train_data, _val_for_fit(y_train_data), 
                    validation_data=(x_valid_data,_val_for_fit(y_valid_data)),
                    epochs=nb_epoch, 
                    batch_size=512
                           # callbacks=[self.checkpointer],
                )
valid_rmspe=calc_predict_rmspe(embedding_model,x_valid_data,y_valid_data)
train_rmspe=calc_predict_rmspe(embedding_model,x_train_data,y_train_data)
print "train_rmspe :{:f},\tvalid_rmspe:{:f}".format(train_rmspe,valid_rmspe)



Train on 786180 samples, validate on 45852 samples
Epoch 1/1
786180/786180 [==============================] - 17s - loss: 0.2957 - val_loss: 0.1850
train_rmspe :0.195616,	valid_rmspe:0.186374


In [46]:
nb_epoch=1

embedding_model=create_embedding_model()
embedding_model.fit(x_train_data, _val_for_fit(y_train_data), 
                    validation_data=(x_valid_data,_val_for_fit(y_valid_data)),
                    epochs=nb_epoch, 
                    batch_size=512
                           # callbacks=[self.checkpointer],
                )
valid_rmspe=calc_predict_rmspe(embedding_model,x_valid_data,y_valid_data)
train_rmspe=calc_predict_rmspe(embedding_model,x_train_data,y_train_data)
print "train_rmspe :{:f},\tvalid_rmspe:{:f}".format(train_rmspe,valid_rmspe)



Train on 786180 samples, validate on 45852 samples
Epoch 1/1
786180/786180 [==============================] - 16s - loss: 0.2848 - val_loss: 0.1823
train_rmspe :0.186483,	valid_rmspe:0.183717
